In [7]:
import subprocess
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" 

Exception: File `'{os.path.dirname(os.getcwd())}\\general_functions\\generalFunctions.ipynb'` not found.

In [ ]:
from pptx import Presentation
import win32com.client as win32
import pandas as pd
import numpy as np
from pathlib import Path
import re
import sys 
win32c = win32.constants
import shutil
import os
from win32com import client
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from pptx.chart.data import CategoryChartData,XyChartData,BubbleChartData
import win32com.client
from pptx.dml.color import RGBColor
from pptx.util import Inches
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_TICK_LABEL_POSITION
from pptx.enum.chart import XL_LABEL_POSITION
from win32com.client import constants as xl
from pptx.enum.chart import XL_CHART_TYPE
import pickle
from pptx.enum.dml import MSO_LINE,MSO_LINE_DASH_STYLE
import time
import itertools
from pptx.enum.chart import XL_AXIS_CROSSES,XL_LEGEND_POSITION
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict 
from io import BytesIO 


## Slide_1 :Market_Trends

In [ ]:
def Totals_Table_Fill(table, list_duplicates, df_totals, cols, slidenum):
    """
    Fills the totals table in the slide.
    
    Parameters:
        table (Table): Table shape in the slide.
        list_duplicates (list): List of duplicate names for identifying slides.
        df_totals (dict): Dictionary of total DataFrames for each duplicate name.
        cols (list): Columns in the DataFrame.
        slidenum (int): Slide number.
        
    Returns:
        Table: Updated table shape.
    """
    for i, row in enumerate(table.rows):
        if i != 0:
            for j, cell in enumerate(row.cells):
                cell.text = str(round(df_totals[list_duplicates[slidenum]][cols].iloc[0, 1:4][j] / 1000000, 1 ))
                cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].runs[0].font.name = 'Nexa Book'
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    return table

def Column_Chart_Fill(chart, scope):
    """
    Fills the column chart in the slide.
    
    Parameters:
        chart (Chart): Chart shape in the slide.
        scope (list): List of scope names.
    """
    client_colors = [RGBColor(0, 80, 75), RGBColor(0, 108, 109), RGBColor(0, 160, 151), RGBColor(126, 202, 196), RGBColor(153, 199, 197), RGBColor(178, 223, 220)]
    gray_colors = [RGBColor(174, 171, 171), RGBColor(166, 166, 166), RGBColor(155, 152, 152), RGBColor(127, 127, 127), RGBColor(87, 85, 85)]

    for i, series in enumerate(chart.series):
        if series.name in scope:
            series.format.fill.solid()
            series.format.fill.fore_color.rgb = client_colors[i if i < len(client_colors) else -1]
        else:
            series.format.fill.solid()
            series.format.fill.fore_color.rgb = gray_colors[i if i < len(gray_colors) else -1]                       
        for j, point in enumerate(series.points):
            data_label = point.data_label
            data_label.has_text_frame = True
            data_label.text_frame.text =  str(round(series.values[j], 1))
            data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)


In [ ]:
def Market_Trends(prs, list_duplicates, modified_df, df_totals, scope, position=0, slide_by='', years = []):
    """
    Generates market trend analysis slides with charts and tables.
    
    Parameters:
        prs (Presentation): PowerPoint presentation object.
        list_duplicates (list): List of duplicate names for identifying slides.
        modified_df (dict): Dictionary of modified DataFrames for each duplicate name.
        df_totals (dict): Dictionary of total DataFrames for each duplicate name.
        scope (list): List of scope names.
        position (int, optional): Position offset for slides. Defaults to 0.
        slide_by (str, optional): Slide grouping criteria. Defaults to ''.
    """
    for slidenum in range(len(list_duplicates)): 
        shapes = prs.slides[slidenum + position].shapes
        charts = []
        tables = []
        title = shapes.title.text
   
        titleNumber = get_shape_number(shapes, "Market Trends Analysis | By Sector | National | Year over Year") #9
        headerNumber = get_shape_number(shapes, "Sector Growth vs. Company Growth (Replace with So What)") #10
        datasourcenum = get_shape_number(shapes, " DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8

        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text = f'Market Trends Analysis | By {slide_by} | ' + list_duplicates[slidenum] + ' | Year over Year'.replace("  "," ")


        

        df= modified_df[list_duplicates[slidenum]].copy()
        volume_cols = [c for c in df.columns.astype(str)if (f'{slide_by}' in c or ('Volume Sales' in c and any(str(year) in c for year in years)))and 'IYA' not in c]
        value_cols = [c for c in df.columns.astype(str) if (f'{slide_by}' in c or ('Value Sales' in c and any(str(year) in c for year in years))) and 'IYA' not in c]
        
        existing_vol_col = set(volume_cols)
        for year in years:
            col_name = f'Volume Sales {year}'
            if col_name not in existing_vol_col:
                df[col_name] = 0
                df_totals[list_duplicates[slidenum]][col_name] = 0
                volume_cols.append(col_name)
                volume_cols = sorted(volume_cols, key=lambda x: (x != f'{slide_by}', x.split(' ')))
        
        
        existing_val_col = set(value_cols)
        for year in years:
            col_name = f'Value Sales {year}'
            if col_name not in existing_val_col:
                df[col_name] = 0
                df_totals[list_duplicates[slidenum]][col_name] = 0
                value_cols.append(col_name)
                value_cols = sorted(value_cols, key=lambda x: (x != f'{slide_by}', x))



        for shape in shapes:
            if shape.has_chart:
                charts.append(shape)
            if shape.has_table:
                tables.append(shape)


        for chartnum in range(2):
            chart = charts[chartnum].chart
            table = tables[chartnum].table
            chart_data = CategoryChartData()
            chart_data.categories = sorted(years)

            # volume_cols = [c for c in modified_df[list_duplicates[slidenum]].columns[modified_df[list_duplicates[slidenum]].columns.str.contains(f'{slide_by}|Volume Sales')]]
            # value_cols = [c for c in modified_df[list_duplicates[slidenum]].columns[(modified_df[list_duplicates[slidenum]].columns.str.contains(f'{slide_by}|Value Sales')) & ~(modified_df[list_duplicates[slidenum]].columns.str.contains('IYA|Total'))]]
            
            if chartnum == 0:    
                for i in range(df.shape[0]):
                    series_name = df[volume_cols].iloc[i, 0]
                    number = df[volume_cols].iloc[i, 1:4] / 1000000  
                    series = chart_data.add_series(series_name, number)
                    chart.replace_data(chart_data)
                    Column_Chart_Fill(chart, scope)
                Totals_Table_Fill(table, list_duplicates, df_totals, volume_cols, slidenum)

            elif chartnum == 1:
                for i in range(modified_df[list_duplicates[slidenum]].shape[0]):
                    series_name = df[value_cols].iloc[i, 0]
                    number = df[value_cols].iloc[i, 1:4] / 1000000      
                    series = chart_data.add_series(series_name, number)
                    chart.replace_data(chart_data)
                    Column_Chart_Fill(chart, scope)
                Totals_Table_Fill(table, list_duplicates, df_totals, value_cols, slidenum)



## Slide_2

In [ ]:
def Column_Chart_Fill_2(chart, scope):
    """
    Fills the column chart in the slide with a monochromatic palette for client brands.
    
    Parameters:
        chart (Chart): Chart shape in the slide.
        scope (list): List of scope names.
    """
    client_colors = [RGBColor(0, 80, 75), RGBColor(0, 108, 109), RGBColor(0, 160, 151), RGBColor(126, 202, 196), RGBColor(153, 199, 197), RGBColor(178, 223, 220)]
    gray_colors = [RGBColor(174, 171, 171), RGBColor(166, 166, 166), RGBColor(155, 152, 152), RGBColor(127, 127, 127), RGBColor(87, 85, 85)]

    names = [series.name for series in chart.series if series.name in scope]
    for i, series in enumerate(chart.series):
        if series.name in scope:
            col = i if len(names) != 1 else 0
            series.format.fill.solid()
            series.format.fill.fore_color.rgb = client_colors[col if col < len(client_colors) else -1]
        else:
            series.format.fill.solid()
            series.format.fill.fore_color.rgb = gray_colors[i if i < len(gray_colors) else -1]

        for j, point in enumerate(series.points):
            data_label = point.data_label
            data_label.has_text_frame = True
            data_label.text_frame.text = str(int(round(series.values[j] * 100, 0))) + '%'
            data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
            data_label.text_frame.paragraphs[0].runs[0].font.name = 'Nexa Book'
            data_label.text_frame.paragraphs[0].runs[0].font.size = Pt(8)

def Totals_Table_Fill_2(table, list_duplicates, df_totals, cols, slidenum):
    """
    Fills the totals table in the slide.
    
    Parameters:
        table (Table): Table shape in the slide.
        list_duplicates (list): List of duplicate names for identifying slides.
        df_totals (dict): Dictionary of total DataFrames for each duplicate name.
        cols (list): Columns in the DataFrame.
        slidenum (int): Slide number.
        
    Returns:
        Table: Updated table shape.
    """
    for i, row in enumerate(table.rows):
        if i != 0:
            for j, cell in enumerate(row.cells):
                if j > 0:
                    cell.text = str(round(df_totals[list_duplicates[slidenum]][cols].iloc[0, 1:4][j - 1] / 1000000, 1))
                cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].runs[0].font.name = 'Nexa Book'
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    return table


In [ ]:
def Market_Concentration(prs, list_duplicates, modified_df, df_totals, scope, position=0, slide_by='', years = []):
    """
    Generates market concentration analysis slides with charts and tables.
    
    Parameters:
        prs (Presentation): PowerPoint presentation object.
        list_duplicates (list): List of duplicate names for identifying slides.
        modified_df (dict): Dictionary of modified DataFrames for each duplicate name.
        df_totals (dict): Dictionary of total DataFrames for each duplicate name.
        scope (list): List of scope names.
        position (int, optional): Position offset for slides. Defaults to 0.
        slide_by (str, optional): Slide grouping criteria. Defaults to ''.
    """
    for slidenum in range(len(list_duplicates)):
        shapes = prs.slides[slidenum + position].shapes
        charts = []
        tables = []
    
            
        titleNumber = get_shape_number(shapes, "Concentration/Market position analysis by manufacturer | Confectionary | National") #9
        headerNumber = get_shape_number(shapes, "Sector Growth vs. Company Growth (Replace with So What)") #10
        datasourcenum = get_shape_number(shapes, "  DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8

        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text = (f'Concentration/Market position analysis | by {slide_by} | ' + list_duplicates[slidenum]).replace("  ", " ")

        
        df= modified_df[list_duplicates[slidenum]].copy()

        volume_share_cols = [c for c in df.columns.astype(str) if (f'{slide_by}' in c or ('Volume Share' in c and any(str(year) in c for year in years))) and 'IYA' not in c]
        value_share_cols = [c for c in df.columns.astype(str) if (f'{slide_by}' in c or ('Value Share' in c and any(str(year) in c for year in years))) and 'IYA' not in c]
        
        volume_cols = [c for c in df.columns.astype(str) if (f'{slide_by}' in c or ('Volume Sales' in c and any(str(year) in c for year in years))) and 'IYA' not in c]
        value_cols = [c for c in df.columns.astype(str) if (f'{slide_by}' in c or ('Value Sales' in c and any(str(year) in c for year in years))) and 'IYA' not in c]


        existing_vol_share = set(volume_share_cols)
        existing_val_share = set(value_share_cols)
        existing_vol_col = set(volume_cols)
        existing_val_col = set(value_cols)

        for year in years:
            col_1 = f'Volume Share {year}'
            if col_1 not in existing_vol_share:
                df[col_1] = 0
                df_totals[list_duplicates[slidenum]][col_1] = 0
                volume_share_cols.append(col_1)
                volume_share_cols = sorted(volume_share_cols, key=lambda x: (x != f'{slide_by}', x.split(' ')))
        
            col_2 = f'Value Share {year}'
            if col_2 not in existing_val_share:
                df[col_2] = 0
                df_totals[list_duplicates[slidenum]][col_2] = 0
                value_share_cols.append(col_2)
                value_share_cols = sorted(value_share_cols, key=lambda x: (x != f'{slide_by}', x))
        
            col_3 = f'Volume Sales {year}'
            if col_3 not in existing_vol_col:
                df[col_3] = 0
                df_totals[list_duplicates[slidenum]][col_3] = 0
                volume_cols.append(col_3)
                volume_cols = sorted(volume_cols, key=lambda x: (x != f'{slide_by}', x.split(' ')))        

            col_4 = f'Value Sales {year}'
            if col_4 not in existing_val_col:
                df[col_4] = 0
                df_totals[list_duplicates[slidenum]][col_4] = 0
                value_cols.append(col_4)
                value_cols = sorted(value_cols, key=lambda x: (x != f'{slide_by}', x))



        for shape in shapes:
            if shape.has_chart:
                charts.append(shape)
            if shape.has_table:
                tables.append(shape)

        for chartnum in [-1, 0]:
            chart = charts[chartnum + 1].chart
            table = tables[chartnum].table
            chart_data = CategoryChartData()
            chart_data.categories = sorted(years)
            
            # volume_cols = [c for c in modified_df[list_duplicates[slidenum]].columns[modified_df[list_duplicates[slidenum]].columns.str.contains(f'{slide_by}|Volume Share')]]
            # value_cols = [c for c in modified_df[list_duplicates[slidenum]].columns[modified_df[list_duplicates[slidenum]].columns.str.contains(f'{slide_by}|Value Share')]]
            
            if chartnum == 0:
                for i in range(df.shape[0]):
                    series_name = df[volume_share_cols].iloc[i, 0]
                    number = df[volume_share_cols].iloc[i, 1:4]
                    chart_data.add_series(series_name, number)
                    chart.replace_data(chart_data)
                    Column_Chart_Fill_2(chart, scope)
                Totals_Table_Fill_2(table, list_duplicates, df_totals, volume_cols, slidenum)
                
            elif chartnum == -1:
                for i in range(df.shape[0]):
                    series_name = df[value_share_cols].iloc[i, 0]
                    number = df[value_share_cols].iloc[i, 1:4]
                    chart_data.add_series(series_name, number)
                    chart.replace_data(chart_data)
                    Column_Chart_Fill_2(chart, scope)
                # print(df_totals[list_duplicates[slidenum]].shape[1], list_duplicates[slidenum], value_cols)    
                Totals_Table_Fill_2(table, list_duplicates, df_totals, value_cols, slidenum)



## Slide_3:Market_Growth

In [ ]:
# def Market_Growth(prs, list_duplicates, modified_df_P12M, dfs_dya,position = 0, slide_by = '',slide_for = ''):
#     """
#         Generates market growth analysis slides with charts and tables.

#         Parameters:
#             prs (Presentation): PowerPoint presentation object.
#             list_duplicates (list): List of duplicate names for identifying slides.
#             modified_df_P12M (dict): Dictionary of modified DataFrames for each duplicate name.
#             dfs_dya (dict): Dictionary of DYA (Year-Ago) DataFrames for each duplicate name.
#             position (int, optional): Position offset for slides. Defaults to 0.
#             slide_by (str, optional): Slide grouping criteria. Defaults to ''.
#             slide_for (str, optional): Specific slide criteria. Defaults to ''.
#     """  

#     for slidenum in range(len(list_duplicates)):
#         shapes = prs.slides[slidenum + position].shapes
#         charts = []
#         tables = []
#         title = shapes.title.text

        
#         titleNumber = get_shape_number(shapes, "Market growth contributors | By Sectors | Confectionary | National | P12M | Index vs YA Value Sales DYA: 3") #9
#         headerNumber = get_shape_number(shapes, "Market growth contributors (Replace with So What)") #10
#         datasourcenum = get_shape_number(shapes, "  DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8

#         if titleNumber is not None:
#             shapes[datasourcenum].text = data_source

#             dya_value = int(dfs_dya[list_duplicates[slidenum]]['Value Sales IYA'] *100 -100)
#             if slide_for:
#                 shapes[titleNumber].text = f'Market growth contributors | By {slide_for} | ' + list_duplicates[slidenum] + ' | P12M | Index vs YA Value Sales | DYA: ' + ('+' if dya_value > 0 else '')+ str(dya_value)+ "%".replace("  "," ")
#             else:
#                 shapes[titleNumber].text = f'Market growth contributors | By {slide_by} | ' + list_duplicates[slidenum] + ' | P12M | Index vs YA Value Sales | DYA: ' +('+' if dya_value > 0 else '')+ str(dya_value)+ "%".replace("  "," ")
                

#         for shape in shapes:
#             if shape.has_chart:
#                 shape_id = shape.shape_id
#                 charts.append(shape)
#             if shape.has_table:
#                 shape_id = shape.shape_id
#                 tables.append(shape)

#         cols = {
#             'C2': 'Growth Contribution',
#             'C1': 'WoB %'
#         }

#         for chartnum in range(len(charts)):
#             chart = charts[chartnum].chart
#             chart_name = charts[chartnum].name
#             chart_type = chart.chart_type
#             chart_data = CategoryChartData()

#             chart_data.categories = modified_df_P12M[list_duplicates[slidenum]][f'{slide_by}'].astype(str).tolist()
#             modified_df_P12M[list_duplicates[slidenum]] = modified_df_P12M[list_duplicates[slidenum]].replace(np.nan,0)
#             if chart_name in cols.keys():
#                 chart_data.add_series(cols[chart_name], modified_df_P12M[list_duplicates[slidenum]][cols[chart_name]])
#                 chart.replace_data(chart_data)

#         table = tables[1].table
        
        
#         num_columns_to_remove = (len(table.columns) - modified_df_P12M[list_duplicates[slidenum]].shape[0]) - 1  # Specify the number of rows to remove from the end
#         for _ in range(num_columns_to_remove):
#             col = table.columns[modified_df_P12M[list_duplicates[slidenum]].shape[0]]
#             remove_col(table, col)

#         table_width = Inches(9.22)  # Specify the desired table height
#         # table.columns[0].width=524500
#         table.columns[0].width=Inches(0.86)

#         total_col_width = table_width - table.columns[0].width
#         num_columns = len(table.columns)-1  # Exclude the first row

#         if num_columns > 0:
#             cell_width = total_col_width / num_columns
#             for col in range(1, table.columns.__len__()):
#                 table.columns[col].width = int(cell_width)
                        
#         data_ = ['Value Sales', 'Value Sales IYA']

#         for i, row in enumerate(table.rows):
#             for j, cell in enumerate(row.cells):
#                 cell.fill.solid()
#                 if j == 0:
#                     if i == 0:
#                         cell.text = cell.text + " " + currency
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
#                         cell.text_frame.paragraphs[0].font.bold = False
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
#                         cell.text_frame.paragraphs[0].font.size = Pt(7)
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                         # "Value Sales\n(‘000’000)"
#                 if j != 0:
#                     if i == 0:
#                         # if sign == "Before":
#                         cell.text = str(round(modified_df_P12M[list_duplicates[slidenum]][[data_[i]]].iloc[j - 1].values[0] / 1000000, 1))
#                         # else:
#                         #     cell.text = str(round(modified_df_P12M[list_duplicates[slidenum]][[data_[i]]].iloc[j - 1].values[0] / 1000000, 1)) +currency
                          
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
#                         cell.text_frame.paragraphs[0].font.bold = False
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
#                         cell.text_frame.paragraphs[0].font.size = Pt(7)
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        

#                     else:
#                         cell.text = str(round(modified_df_P12M[list_duplicates[slidenum]][[data_[i]]].iloc[j - 1].values[0] * 100))
#                         if int(cell.text) <= 90:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192, 0, 0)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         elif int(cell.text) <= 98:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 128, 128)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         elif int(cell.text) >= 110:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 160, 151)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         elif int(cell.text) >= 102:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(178, 223, 220)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                         else:
#                             cell.text_frame.paragraphs[0].font.size = Pt(7)
#                             cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                             cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

#                 else:
#                     cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
#                     cell.text_frame.paragraphs[0].font.size = Pt(7)
#                     cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER


## Slide_4:ValueSales_AvgPrice

In [ ]:
# def ValueSales_AvgPrice(prs, list_duplicates, modified_df_P12M, modified_df_clients,manuf_brand_list,market_list, position = 0, slide_by = '', slide_for = ''):
#     """
#         Generates value sales and average price analysis slides with charts and tables.
 
#         Parameters:
#             prs (Presentation): PowerPoint presentation object.
#             list_duplicates (list): List of duplicate names for identifying slides.
#             modified_df_P12M (dict): Dictionary of modified DataFrames for each duplicate name.
#             modified_df_clients (dict): Dictionary of client-specific DataFrames.
#             manuf_brand_list (list): List of manufacturer brand names.
#             market_list (list): List of market names.
#             position (int, optional): Position offset for slides. Defaults to 0.
#             slide_by (str, optional): Slide grouping criteria. Defaults to ''.
#             slide_for (str, optional): Specific slide criteria. Defaults to ''.
#     """    
#     global y
#     for slidenum in range(len(list_duplicates)):
#         # Extract client key, market, and manufacturer brand
#         client_key = list(modified_df_clients.keys())[slidenum]
#         market = list(set(client_key.split(' | ')).intersection(set(market_list)))[0]
#         manuf_brand =  list(set(client_key.split(' | ')).intersection(set(manuf_brand_list)))[0]
#          # Find the corresponding P12M key
#         P12M_key = [key for key in modified_df_P12M.keys() if market in key.split(' | ')][0]
       
#         # Get the shapes of the current slide
#         shapes = prs.slides[slidenum+ position].shapes
#         charts = []
#         tables = []
#         title = shapes.title.text

        
#         titleNumber = get_shape_number(shapes, "Sectors Value Sales & Avg Price Per Kg | National | P12M") #9
#         headerNumber = get_shape_number(shapes, "Sectors Value Sales & Avg Price Per Kg (Replace with So What)") #10
#         datasourcenum = get_shape_number(shapes, "  DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8
#         AvgPriceNum = get_shape_number(shapes, "Avg Price/Kg")


#         if titleNumber is not None:
#             shapes[datasourcenum].text = data_source
#             shapes[AvgPriceNum].text = shapes[AvgPriceNum].text + f" ({currency.strip()})"
#             shapes[AvgPriceNum].text_frame.paragraphs[0].font.size = Pt(7)
#             shapes[AvgPriceNum].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
#             shapes[AvgPriceNum].text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
            
#             if slide_for:
#                 shapes[titleNumber].text = f'{slide_for} Value Sales & Avg Price Per Kg | '+ list_duplicates[slidenum] +' | P12M '.replace("  "," ")
#                 shapes[headerNumber].text = f'{slide_for} Value Sales & Avg Price Per Kg (Replace with So What)'
#             else:
#                 shapes[titleNumber].text = f'{slide_by} Value Sales & Avg Price Per Kg | '+ list_duplicates[slidenum] +' | P12M '.replace("  "," ")
#                 shapes[headerNumber].text = f'{slide_by} Value Sales & Avg Price Per Kg (Replace with So What)'

#         # Collect charts and tables from the shapes
#         for shape in shapes:
#             if shape.has_chart:
#                 shape_id = shape.shape_id
#                 charts.append(shape)
#             if shape.has_table:
#                 shape_id = shape.shape_id
#                 tables.append(shape)
 
#         cols = {
#             'C1': 'Value Sales IYA',
#             'C2':'Value Sales'    
#         }
 
#         for chartnum in range(len(charts)):
#             chart = charts[chartnum].chart
#             chart_name = charts[chartnum].name
#             if chart_name == 'C2':    
#                 chart_data = CategoryChartData()
#                 chart_data.categories = modified_df_P12M[P12M_key][f'{slide_by}'].astype(str).tolist()
 
#                 chart_data.add_series("Value Sales", (modified_df_P12M[P12M_key]['Value Sales']/1000000))
 
#                 chart_data.add_series("Av Price/KG", modified_df_P12M[P12M_key]['Av Price/KG'])
#                 value_axis = chart.value_axis
#                 # value_axis.tick_labels.number_format = f'{currency}#,##0.00' if sign == 'Before' else f'#,##0.00{currency}'
#                 value_axis.auto_axis = True
#                 for i,series in enumerate(chart.series):
#                     if series.name=='Av Price/KG':
#                         for j, point in enumerate(series.points):
#                             data_label = point.data_label
#                             data_label.position=XL_LABEL_POSITION.ABOVE
#                             data_label.has_text_frame = True
#                             # if sign == "Before":
#                             data_label.text_frame.text = (str(round(series.values[j], decimals)) +'0' if len(str(round(series.values[j], decimals)).split('.')[1])==1 else str(round(series.values[j], decimals)))
#                             # else:
#                             #     data_label.text_frame.text = (str(round(series.values[j], decimals)) +'0' + f'{currency} ' if len(str(round(series.values[j], decimals)).split('.')[1])==1 else str(round(series.values[j], decimals)) + f'{currency} ')
#                 chart.replace_data(chart_data)
 
#             if chart_name == 'C1':
 
#                 chart_data = CategoryChartData()
 
#                 chart_data.categories = modified_df_P12M[P12M_key][f'{slide_by}'].astype(str).tolist()
#                 chart_data.add_series('Value Sales IYA', (modified_df_P12M[P12M_key]['Value Sales IYA']))
#                 value_axis = chart.value_axis
#                 value_axis.auto_axis = True
#                 value_axis.maximum_scale = None
#                 value_axis.minimum_scale = None
#                 chart.replace_data(chart_data)
 
#         table = tables[0].table
 
#         num_columns_to_remove = (len(table.columns) - modified_df_P12M[P12M_key].T.shape[1])-1  # Specify the number of rows to remove from the end
 
#         rightSahpe=len(table.columns)-num_columns_to_remove
#         while len(table.columns)!=rightSahpe:
#             col = table.columns[0]
#             remove_col(table, col)
           
#         table_width = Inches(8.71)  # Specify the desired table height
#         table.columns[0].width=Inches(1.32)
#         table.height = Inches(1.41)
 
#         total_col_width = table_width - table.columns[0].width
#         num_columns = len(table.columns)-1  # Exclude the first row
   
#         if num_columns > 0:
#             cell_width = total_col_width / num_columns
#             for col in range(1, table.columns.__len__()):
#                 table.columns[col].width = int(cell_width)        
               
#         for i in range(len(table.columns)-1):
#             table.columns[i+1].width=int(total_col_width/num_columns)
   
#         modified_P12M_clients = {}
#         modified_P12M_clients[list_duplicates[slidenum]] = modified_df_P12M[P12M_key][[f'{slide_by}', 'WoB %', "Relative Price"]].merge(modified_df_clients[client_key][[f'{slide_by}', 'Value Share','Value Share DYA', 'Av Price/KG']], on = f'{slide_by}', how = 'left')
#         modified_P12M_clients[list_duplicates[slidenum]] = modified_P12M_clients[list_duplicates[slidenum]].replace(np.nan,0)
#         modified_P12M_clients[list_duplicates[slidenum]]["Relative Price"] = modified_P12M_clients[list_duplicates[slidenum]]["Relative Price"]*100
           
#         modified_P12M_clients[list_duplicates[slidenum]].columns = modified_P12M_clients[list_duplicates[slidenum]].columns.str.replace('WoB %', f"{slide_by} WoB %").str.replace( "Relative Price",f"{slide_by} Avg Price Index").str.replace( "Value Share", f"{manuf_brand} Share").str.replace("Av Price/KG",f"{manuf_brand} Avg Price/KG ({currency.strip()})")      
   
       
#         for i, row in enumerate(table.rows):
 
#             for j, cell in enumerate(row.cells):
#                 if 'avg' in modified_P12M_clients[list_duplicates[slidenum]].T.index[i+1].strip().lower():
#                     if i!=3:
#                         cell.text = str(int(round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+1, j-1]),0)))
                   
                       
#                 elif i==0 :
#                     cell.text = str((round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+1,j-1])*100,1)))+'%'
 
#                 elif i==2:
#                     shares_value = round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i + 1, j - 1]) * 100, 1)
#                     shares = '' if shares_value == 0 else str(shares_value) + '%'
                   
#                     sharesdya_value = round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i + 2, j - 1]) * 100, 1)
#                     sharesdya = '' if sharesdya_value == 0 else str(sharesdya_value) + '%'
#                     # Clear the existing cell text first to avoid duplication
#                     cell.text = ""
#                     # Set the shares part
#                     paragraph = cell.text_frame.paragraphs[0]
#                     paragraph.clear()  # Clear any existing text in the paragraph
#                     shares_run = paragraph.add_run()
#                     shares_run.text = shares
#                     shares_run.font.size = Pt(7)
#                     shares_run.font.name = 'Nexa Book'
                       
#                     if sharesdya:
#                         pipe_run = paragraph.add_run()
#                         pipe_run.text = " | "  # Add the pipe as a separate run
#                         pipe_run.font.size = Pt(7)
#                         pipe_run.font.name = 'Nexa Book'
#                         pipe_run.font.color.rgb = RGBColor(87, 85, 85)  # Default color for the pipe
 
#                         # Now set the sharesdya part
#                         sharesdya_run = paragraph.add_run()
#                         sharesdya_run.text = sharesdya
#                         sharesdya_run.font.size = Pt(7)
#                         sharesdya_run.font.name = 'Nexa Book'
#                         # Apply color formatting only to the sharesdya value
#                         if sharesdya_value > 0.2:
#                             sharesdya_run.font.color.rgb = RGBColor(0, 160, 151)  # Green for positive values
#                         elif sharesdya_value < -0.2:
#                             sharesdya_run.font.color.rgb = RGBColor(192, 0, 0)  # Red for negative values
#                         else:
#                             sharesdya_run.font.color.rgb = RGBColor(87, 85, 85)  # Default black for zero
#                 elif i==3:
#                     # if sign == "Before":
#                     #     sales='' if round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]),decimals)==0 else f'{currency}'+str(round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]),decimals))
#                     # else:
#                     sales = '' if round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]), decimals) == 0 else str(round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]), decimals)) #+ f'{currency}'
#                     sales = '' if sales=='' else sales+'0' if ('.' in sales and len(sales.split('.')[1])==1) else sales+'.'+'0'*decimals if '.' not in sales else sales
#                     cell.text = sales
                   
#                 cell.fill.solid()
#                 cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                 cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
#                 cell.text_frame.paragraphs[0].font.size = Pt(7)
#                 cell.fill.fore_color.rgb=RGBColor(242,242,242)
#                 if  (cell.text=="0%")or (cell.text=="0.0%"):  
#                     cell.text = ''
#                 if j == 0:
#                     text_value = str(modified_P12M_clients[list_duplicates[slidenum]].T.index[i+1])
                   
#                     # Skip the "Edgewell Share DYA" entry
#                     if text_value == f"{manuf_brand} Share DYA":
#                         text_value = str(modified_P12M_clients[list_duplicates[slidenum]].T.index[i+2])
                   
#                     # print(text_value)
#                     cell.text = text_value
#                     cell.text_frame.paragraphs[0].font.size = Pt(7)
#                     cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
#                     cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255,255,255)
#                     cell.fill.fore_color.rgb=RGBColor(0,160,151)
 

In [ ]:
def ValueSales_AvgPrice(prs, list_duplicates, modified_df_P12M, modified_df_clients,manuf_brand_list,market_list, position = 0, slide_by = '', slide_for = ''):
    """
        Generates value sales and average price analysis slides with charts and tables.
 
        Parameters:
            prs (Presentation): PowerPoint presentation object.
            list_duplicates (list): List of duplicate names for identifying slides.
            modified_df_P12M (dict): Dictionary of modified DataFrames for each duplicate name.
            modified_df_clients (dict): Dictionary of client-specific DataFrames.
            manuf_brand_list (list): List of manufacturer brand names.
            market_list (list): List of market names.
            position (int, optional): Position offset for slides. Defaults to 0.
            slide_by (str, optional): Slide grouping criteria. Defaults to ''.
            slide_for (str, optional): Specific slide criteria. Defaults to ''.
    """    
    global y
    for slidenum in range(len(list_duplicates)):
        # Extract client key, market, and manufacturer brand
        client_key = list(modified_df_clients.keys())[slidenum]
        market = list(set(client_key.split(' | ')).intersection(set(market_list)))[0]
        manuf_brand =  list(set(client_key.split(' | ')).intersection(set(manuf_brand_list)))[0]
         # Find the corresponding P12M key
        P12M_key = [key for key in modified_df_P12M.keys() if market in key.split(' | ')][0]
       
        # Get the shapes of the current slide
        shapes = prs.slides[slidenum+ position].shapes
        charts = []
        tables = []
        title = shapes.title.text

        
        titleNumber = get_shape_number(shapes, "Sectors Value Sales & Avg Price Per Kg | National | P12M") #9
        headerNumber = get_shape_number(shapes, "Sectors Value Sales & Avg Price Per Kg (Replace with So What)") #10
        datasourcenum = get_shape_number(shapes, "  DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8
        AvgPriceNum = get_shape_number(shapes, "Avg Price/Kg")


        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[AvgPriceNum].text = shapes[AvgPriceNum].text + f" ({currency.strip()})"
            shapes[AvgPriceNum].text_frame.paragraphs[0].font.size = Pt(7)
            shapes[AvgPriceNum].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
            shapes[AvgPriceNum].text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
            
            if slide_for:
                shapes[titleNumber].text = f'{slide_for} Value Sales & Avg Price Per Vol | '+ list_duplicates[slidenum] +' | P12M '.replace("  "," ")
                shapes[headerNumber].text = f'{slide_for} Value Sales & Avg Price Per Vol (Replace with So What)'
            else:
                shapes[titleNumber].text = f'{slide_by} Value Sales & Avg Price Per Vol | '+ list_duplicates[slidenum] +' | P12M '.replace("  "," ")
                shapes[headerNumber].text = f'{slide_by} Value Sales & Avg Price Per Vol (Replace with So What)'

        # Collect charts and tables from the shapes
        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                charts.append(shape)
            if shape.has_table:
                shape_id = shape.shape_id
                tables.append(shape)
 
        cols = {
            'C1': 'Value Sales IYA',
            'C2':'Value Sales'    
        }
        modified_df_P12M[P12M_key] = modified_df_P12M[P12M_key].replace(np.nan, 0)
 
        for chartnum in range(len(charts)):
            chart = charts[chartnum].chart
            chart_name = charts[chartnum].name
            if chart_name == 'C2':    

                chart_data = CategoryChartData()
                chart_data.categories = modified_df_P12M[P12M_key][f'{slide_by}'].astype(str).tolist()
 
                chart_data.add_series("Value Sales", (modified_df_P12M[P12M_key]['Value Sales']/1000000))
 
                chart_data.add_series("Av Price/KG", modified_df_P12M[P12M_key]['Av Price/KG'])
                value_axis = chart.value_axis
                # value_axis.tick_labels.number_format = f'{currency}#,##0.00' if sign == 'Before' else f'#,##0.00{currency}'
                value_axis.auto_axis = True
                for i,series in enumerate(chart.series):
                    if series.name=='Av Price/KG':
                        for j, point in enumerate(series.points):
                            data_label = point.data_label
                            data_label.position=XL_LABEL_POSITION.ABOVE
                            data_label.has_text_frame = True
                            # if sign == "Before":
                            data_label.text_frame.text = (str(round(series.values[j], decimals)) +'0' if len(str(round(series.values[j], decimals)).split('.')[1])==1 else str(round(series.values[j], decimals)))
                            # else:
                            #     data_label.text_frame.text = (str(round(series.values[j], decimals)) +'0' + f'{currency} ' if len(str(round(series.values[j], decimals)).split('.')[1])==1 else str(round(series.values[j], decimals)) + f'{currency} ')
                chart.replace_data(chart_data)
 
            if chart_name == 'C1':
 
                chart_data = CategoryChartData()
 
                chart_data.categories = modified_df_P12M[P12M_key][f'{slide_by}'].astype(str).tolist()
                chart_data.add_series('Value Sales IYA', (modified_df_P12M[P12M_key]['Value Sales IYA']))
                value_axis = chart.value_axis
                value_axis.auto_axis = True
                value_axis.maximum_scale = None
                value_axis.minimum_scale = None
                chart.replace_data(chart_data)
 
        table = tables[0].table
 
        num_columns_to_remove = (len(table.columns) - modified_df_P12M[P12M_key].T.shape[1])-1  # Specify the number of rows to remove from the end
 
        rightSahpe=len(table.columns)-num_columns_to_remove
        
        if rightSahpe < 0:
    # Defensive: Cannot have negative columns to keep
            rightSahpe = 0
 
        while len(table.columns) > rightSahpe and len(table.columns) > 0:
            col = table.columns[0]
            remove_col(table, col)
            
        table_width = Inches(9.44)  # Specify the desired table height
        table.columns[0].width=Inches(1.28)
        table.height = Inches(1.43)
 
        total_col_width = table_width - table.columns[0].width
        num_columns = len(table.columns)-1  # Exclude the first row
   
        if num_columns > 0:
            cell_width = total_col_width / num_columns
            for col in range(1, table.columns.__len__()):
                table.columns[col].width = int(cell_width)        
               
        for i in range(len(table.columns)-1):
            table.columns[i+1].width=int(total_col_width/num_columns)
   
        modified_P12M_clients = {}
        modified_P12M_clients[list_duplicates[slidenum]] = modified_df_P12M[P12M_key][[f'{slide_by}', "Growth Contribution",'WoB %', "Relative Price"]].merge(modified_df_clients[client_key][[f'{slide_by}', 'Value Share','Value Share DYA', 'Av Price/KG']], on = f'{slide_by}', how = 'left')
        modified_P12M_clients[list_duplicates[slidenum]] = modified_P12M_clients[list_duplicates[slidenum]].replace(np.nan, "")
        # print(modified_P12M_clients[list_duplicates[slidenum]])
        # modified_P12M_clients[list_duplicates[slidenum]] = modified_P12M_clients[list_duplicates[slidenum]].replace(np.nan,0)
        modified_P12M_clients[list_duplicates[slidenum]]["Relative Price"] = modified_P12M_clients[list_duplicates[slidenum]]["Relative Price"]*100
 
           
        modified_P12M_clients[list_duplicates[slidenum]].columns = modified_P12M_clients[list_duplicates[slidenum]].columns.str.replace('WoB %', f"{slide_by} WoB %").str.replace( "Relative Price",f"{slide_by} Avg Price Index").str.replace( "Value Share", f"{manuf_brand} Share").str.replace("Av Price/KG",f"{manuf_brand} Avg Price/Vol ({currency.strip()})")      
   
       
        for i, row in enumerate(table.rows):
 
            for j, cell in enumerate(row.cells):
                if 'avg' in modified_P12M_clients[list_duplicates[slidenum]].T.index[i+1].strip().lower():
                    if i!=4:
                        cell.text = str(int(round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+1, j-1]),0)))
                   
                elif i==0 :
                    cell.text = str((round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+1,j-1])*100,1)))+'%'

                elif i==1 :
                    cell.text = str((round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+1,j-1])*100,1)))+'%'
 
                elif i==3:
                    # shares_value = round(float(modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i + 1, j - 1]) * 100, 1)
                    # shares = '0.0%' if shares_value == 0 else str(shares_value) + '%'
                    raw_val = modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i + 1, j - 1]

                    if pd.isna(raw_val) or raw_val == '':
                        shares = ''
                    else:
                        shares_value = round(float(raw_val) * 100, 1)
                        shares = '0.0%' if shares_value == 0 else str(shares_value) + '%'
                   
                    share_dya = modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i + 2, j - 1]
                    if pd.isna(share_dya) or share_dya == '':
                        sharesdya = ''
                    else:
                        # shar_dya = modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i + 2, j - 1] * 100
                        sharesdya_value = round(float(share_dya) * 100, 1)
                        # sharesdya = '' if sharesdya_value == 0 else str(sharesdya_value) + '%'
                        sharesdya = '0.0%' if sharesdya_value == 0 else f"+{sharesdya_value}%" if sharesdya_value > 0 else f"{sharesdya_value}%"
 
                    # Clear the existing cell text first to avoid duplication
                    cell.text = ""
                    # Set the shares part
                    paragraph = cell.text_frame.paragraphs[0]
                    paragraph.clear()  # Clear any existing text in the paragraph
                    shares_run = paragraph.add_run()
                    shares_run.text = shares
                    shares_run.font.size = Pt(7)
                    shares_run.font.name = 'Nexa Book'
                       
                    if sharesdya:
                        pipe_run = paragraph.add_run()
                        pipe_run.text = " | "  # Add the pipe as a separate run
                        pipe_run.font.size = Pt(7)
                        pipe_run.font.name = 'Nexa Book'
                        pipe_run.font.color.rgb = RGBColor(87, 85, 85)  # Default color for the pipe
 
                        # Now set the sharesdya part
                        sharesdya_run = paragraph.add_run()
                        sharesdya_run.text = sharesdya
                        sharesdya_run.font.size = Pt(7)
                        sharesdya_run.font.name = 'Nexa Book'
                        # Apply color formatting only to the sharesdya value
                        if sharesdya_value > 0.2:
                            sharesdya_run.font.color.rgb = RGBColor(0, 160, 151)  # Green for positive values
                        elif sharesdya_value < -0.2:
                            sharesdya_run.font.color.rgb = RGBColor(192, 0, 0)  # Red for negative values
                        else:
                            sharesdya_run.font.color.rgb = RGBColor(87, 85, 85)  # Default black for zero
                elif i==4:
                    # if sign == "Before":
                    #     sales='' if round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]),decimals)==0 else f'{currency}'+str(round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]),decimals))
                    # else:
                    val = modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]

                    if pd.isna(val) or val == '':
                        sales = ''
                    else:
                        val = float(val)
                        rounded_val = round(val, decimals)
                        if rounded_val == 0:
                            sales = ''
                        else:
                            sales = str(rounded_val)
                            if '.' in sales and len(sales.split('.')[1]) == 1:
                                sales += '0'
                            elif '.' not in sales:
                                sales += '.' + '0' * decimals
                    # sales = '' if round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]), decimals) == 0 else str(round((modified_P12M_clients[list_duplicates[slidenum]].T.iloc[i+2, j-1]), decimals)) #+ f'{currency}'
                    sales = '' if sales=='' else sales+'0' if ('.' in sales and len(sales.split('.')[1])==1) else sales+'.'+'0'*decimals if '.' not in sales else sales
                    cell.text = sales
                   
                cell.fill.solid()
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                cell.text_frame.paragraphs[0].font.size = Pt(7)
                cell.fill.fore_color.rgb=RGBColor(242,242,242)
                if  (cell.text=="0%")or (cell.text=="0.0%"):  
                    cell.text = ''
                if j == 0:
                    text_value = str(modified_P12M_clients[list_duplicates[slidenum]].T.index[i+1])
                    # Skip the "Edgewell Share DYA" entry
                    if text_value == f"{manuf_brand} Share | DYA DYA":
                        text_value = str(modified_P12M_clients[list_duplicates[slidenum]].T.index[i+2])
                   
                    # print(text_value)
                    cell.text = text_value
                    cell.text_frame.paragraphs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255,255,255)
                    cell.fill.fore_color.rgb=RGBColor(0,160,151)
 

## Slide_5:Share_GrowthBy Manufacturer/Brands

In [ ]:
def Share_Growth(prs, list_duplicates, modified_df_P12M_T1, modified_df_P12M_T2, position = 0):
    """
    Generates share and growth analysis slides with charts and tables.

    Parameters:
        prs (Presentation): PowerPoint presentation object.
        list_duplicates (list): List of duplicate names for identifying slides.
        modified_df_P12M_T1 (dict): Dictionary of modified DataFrames (T1) for each duplicate name.
        modified_df_P12M_T2 (dict): Dictionary of modified DataFrames (T2) for each duplicate name.
        position (int, optional): Position offset for slides. Defaults to 0.
    """
    for slidenum in range(len(list_duplicates)):
        shapes = prs.slides[slidenum +position].shapes
        charts = []
        tables = []
        title = shapes.title.text
        
        titleNumber = get_shape_number(shapes, "Share and Growth By Manufacturer/Brands | Total Category | National") #9
        headerNumber = get_shape_number(shapes, "Share and Growth by Manufacturer/Brands (Replace with So What)") #10
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022") #8

        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = 'Share and Growth By Companies/Brands | ' + list_duplicates[slidenum]
            # shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'


        for shape in shapes:
            if shape.has_chart:
                charts.append(shape)

            if shape.has_table:
                tables.append(shape)

        # cols = {
        #     'T1-C1': 'Share DYA LY',
        #     'T1-C2': 'Value Share P12M',
        #     'T1-C3': 'Share DYA P12M'
        # }
        cols = {
            1: 'Share DYA LY',
            0: 'Value Share P12M',
            3: 'Share DYA P12M'
        }
        # chartNum = {0:1,1:0,2:3,3:2,4:4}


        columns_vals = ['Volume Sales', 'Value Sales', 'Volume Share', 'Value Share', 'Share DYA', 'Av Price/KG']

        for year in ["P12M", "LY"]:
            for col in columns_vals:
                col_name = f'{col} {year}'
                if col_name not in modified_df_P12M_T1[list_duplicates[slidenum]].columns:
                    modified_df_P12M_T1[list_duplicates[slidenum]][col_name] = 0    
                    
        # Update T1 charts with data
        for chartnum in [0,1,3]:
            chart = charts[chartnum].chart
            chart_data = CategoryChartData()
            chart_data.categories = ['']
 
            col_name = cols[chartnum]
            chart_data.add_series(col_name, modified_df_P12M_T1[list_duplicates[slidenum]][['Share DYA LY', 'Value Share P12M', 'Share DYA P12M']][col_name])
            chart.replace_data(chart_data)
 
            try:
                series = chart.series[0]
                series.data_labels.show_value = True
                if col_name in ['Share DYA LY', 'Share DYA P12M']:
                    series.data_labels.number_format = '+0.0%;-0.0%;0.0%' 
                else:
                    series.data_labels.number_format = '0.0%' 
            except:
                pass
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 0,"Top Manufacturers")
 
                worksheet.write_column(1, 0, modified_df_P12M_T1[list_duplicates[slidenum]][f'{ManufOrTopC}'])
 
            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        column_indices = [0, 4]
        
        # Update T1 table with data
        for tablenum in range(len(tables)):
            if tables[tablenum].name == 'T1':
                table = tables[tablenum].table
                is_first_row = True  
                num_rows_to_remove = len(table.rows) - len(modified_df_P12M_T1[list_duplicates[slidenum]])-1  # Specify the number of rows to remove from the end

                # Remove rows from the end of the table
                for _ in range(num_rows_to_remove):
                    if len(table.rows) > 1:  # Skip removing the first row if there is more than one row
                        row = table.rows[1]
                        remove_row(table, row)

                table_height = Inches(3.94)  # Specify the desired table height
                total_row_height = table_height - table.rows[0].height
                num_rows = len(table.rows)-1  # Exclude the first row
                if num_rows > 0:
                    cell_height = total_row_height / num_rows
                    for row in range(1, table.rows.__len__()):
                        table.rows[row].height = int(cell_height)

                for i, row in enumerate(table.rows):
                    for j, cell in enumerate(row.cells):
                        if i == 0:
                            if j ==4:
                                cell.text = f"Avg Price /Vol ({currency.strip()})"
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                                cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                        elif j in column_indices:
                            modified_df_P12M_c1 = {}
                            modified_df_P12M_c1[list_duplicates[slidenum]] = modified_df_P12M_T1[list_duplicates[slidenum]].copy()
                            column_index = column_indices.index(j)
                            modified_df_P12M_c1[list_duplicates[slidenum]]['Av Price/KG P12M'] = round(modified_df_P12M_c1[list_duplicates[slidenum]]['Av Price/KG P12M'].astype(float) * 100,decimals)
                            cell.text = str(modified_df_P12M_c1[list_duplicates[slidenum]][[f"{ManufOrTopC}", 'Av Price/KG P12M']].iloc[i - 1, column_index])
                            cell.fill.solid()
                            cell.fill.fore_color.rgb = RGBColor(255, 255, 255)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                            cell.text_frame.paragraphs[0].font.size = Pt(8)
                            if j == 0:
                                cell.text_frame.paragraphs[0].font.size = Pt(9)
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            if j == 4:
                                cell.text =str(round(float(cell.text) / 100, 2))
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
 
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
      
    # #### 2nd table.                   

        # cols = {
        #     'T2-C1': 'Value Share',
        #     'T2-C2': 'Share DYA'
        # }
        cols = {
            2: 'Value Share',
            4: 'Share DYA'
        }
        # chartNum = {0:1,1:0,2:3,3:2,4:4}
        for chartnum in [2,4]:
            chart = charts[chartnum].chart
            chart_type = chart.chart_type
            chart_data = CategoryChartData()
            chart_data.categories = ['']
 
            col_name = cols[chartnum]
            chart_data.add_series(cols[chartnum], modified_df_P12M_T2[list_duplicates[slidenum]][['Value Share', 'Share DYA']][cols[chartnum]])
            chart.replace_data(chart_data)
            try:
                series = chart.series[0]
                series.data_labels.show_value = True
                if col_name  == 'Share DYA':
                    series.data_labels.number_format = '+0.0%;-0.0%;0.0%'
                else:
                    series.data_labels.number_format = '0.0%'
            except:
                pass
            xlsx_file=BytesIO()
 
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 0,f'{BrandOrTopB}')
 
                worksheet.write_column(1, 0, modified_df_P12M_T2[list_duplicates[slidenum]][f'{BrandOrTopB}'])
 
            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        column_indices = [0, 3, 4]

        for tablenum in range(len(tables)):
            if tables[tablenum].name == 'T2':
                table = tables[tablenum].table
                is_first_row = True  # Flag to track the first row

                num_rows_to_remove = len(table.rows) - len(modified_df_P12M_T2[list_duplicates[slidenum]])-1  # Specify the number of rows to remove from the end

                for _ in range(num_rows_to_remove):
                    if len(table.rows) > 1:  # Skip removing the first row if there is more than one row
                        row = table.rows[1]
                        remove_row(table, row)

                table_height = Inches(3.94)  # Specify the desired table height
                total_row_height = table_height - table.rows[0].height
                num_rows = len(table.rows)-1  # Exclude the first row
                if num_rows > 0:
                    cell_height = total_row_height / num_rows
                    for row in range(1, table.rows.__len__()):
                        table.rows[row].height = int(cell_height)

                for i, row in enumerate(table.rows):
                    for j, cell in enumerate(row.cells):
                        if i == 0:
                            if j ==3:
                                cell.text = f"Avg Price /Vol ({currency.strip()})"
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                                cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                        elif j in column_indices:

                            modified_df_P12M_c2 = {}
                            modified_df_P12M_c2[list_duplicates[slidenum]] = modified_df_P12M_T2[list_duplicates[slidenum]].copy()
                            column_index = column_indices.index(j)
                            modified_df_P12M_c2[list_duplicates[slidenum]]['Av Price/KG'] = round(modified_df_P12M_c2[list_duplicates[slidenum]]['Av Price/KG'].astype(float)*100,decimals)
                            modified_df_P12M_c2[list_duplicates[slidenum]]['IYA Price/KG'] = round(modified_df_P12M_c2[list_duplicates[slidenum]]['IYA Price/KG'].astype(float)*100,decimals)
                            cell.text = str(modified_df_P12M_c2[list_duplicates[slidenum]][[f'{BrandOrTopB}',  'Av Price/KG', 'IYA Price/KG']].iloc[i - 1, column_index])
                            #                         cell.fill.solid()
    #                         cell.fill.fore_color.rgb = RGBColor(229, 244, 243)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                            cell.text_frame.paragraphs[0].font.size = Pt(8) if isinstance(i, int) else Pt(9)
                            if j == 0:
                                cell.text_frame.paragraphs[0].font.size = Pt(9)
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'

                            if j == 3:
                                cell.text =str(round(float(cell.text) / 100, decimals))
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

                            if j == 4:
                                cell.text=str(int(round(float(cell.text),0)))

                                if int(float(cell.text)) <= 90:
                                    cell.text = cell.text
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192,0,0)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    
                                elif int(float(cell.text)) <=98:
                                    cell.text = cell.text
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255,128,128)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    
                                elif int(float(cell.text)) >=110:
                                    cell.text = cell.text
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    
                                elif int(float(cell.text)) >=102:
                                    cell.text = cell.text
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(178,223,220)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'

                                else:
                                    cell.text = cell.text
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    

##  Slides_6,7,8:Share and Growth By Manufacturer  Or by Brand


In [11]:
def Share_Growth_Separately(prs, list_duplicates, modified_df_P12M, position = 0,slide_by = '',list_tables=[],first_col=""):
    for slidenum in range(len(list_duplicates)):
        shapes = prs.slides[slidenum + position].shapes
        charts = []
        tables = []
    

        titleNumber = get_shape_number(shapes, "Share and Growth By Manufacturer | By Sector | National | P12M") #9
        headerNumber = get_shape_number(shapes, "Segment Leadership Analysis (Replace with So What)") #10
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022") #8
        print(list_duplicates[slidenum])
        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = f'Share and Growth By {slide_by} | ' + list_duplicates[slidenum] +' | P12M'
            shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        try:
            names = {7: 'T1',
                3 :'T3', 
                31 :'T2'}
            char_names = {12:'T1-C1',
                        27:'T3-C1',
                        38:'T2-C1'}
            cols = {
            'T1-C1':f'{list_tables[0]} | {list_duplicates[slidenum]}',
            'T3-C1':f'{list_tables[1]} | {list_duplicates[slidenum]}',
            'T2-C1':f'{list_tables[2]} | {list_duplicates[slidenum]}' 
        }
            
            tabl = {'T1':f'{list_tables[0]} | {list_duplicates[slidenum]}',
                    'T3':f'{list_tables[1]} | {list_duplicates[slidenum]}',
                    'T2':f'{list_tables[2]} | {list_duplicates[slidenum]}'
            }
        except:
            try:
                names = {7: 'T1',
                    31 :'T2'
                    }
                char_names = {12:'T1-C1',
                38:'T2-C1'}
                cols = {
                'T1-C1':f'{list_tables[0]} | {list_duplicates[slidenum]}',
                'T2-C1':f'{list_tables[1]} | {list_duplicates[slidenum]}'
            }

                tabl = {'T1':f'{list_tables[0]} | {list_duplicates[slidenum]}',
                        'T2':f'{list_tables[1]} | {list_duplicates[slidenum]}'   
                }
            except:
                names = {7: 'T1'
                    }
                char_names = {12:'T1-C1'}
                cols = {
                'T1-C1':f'{list_tables[0]} | {list_duplicates[slidenum]}'
            }

                tabl = {'T1':f'{list_tables[0]} | {list_duplicates[slidenum]}'   
                }
        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                if shape_id in char_names:
                    shape.name = char_names[shape_id]
                else:
                    if shape_id in names:
                        shape.name = names[shape_id] + "-C1"
                charts.append(shape)

            if shape.has_table:
                shape_id = shape.shape_id
                if shape_id in names:
                    shape.name = names[shape_id]
                tables.append(shape)

    
                
                
        for chartnum in range(len(charts)):
            chart = charts[chartnum].chart
           
            chart_name = charts[chartnum].name
            chart_type = chart.chart_type
            chart_data = CategoryChartData()
            chart_data.categories = ['']
    
            if chart_name in cols.keys():
                chart_data.add_series(cols[chart_name], modified_df_P12M[cols[chart_name]]['Value Share P12M'])
                chart.replace_data(chart_data)
                xlsx_file=BytesIO()

                with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                    chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                    worksheet.write(0, 0,first_col)
                    print(chart_name, cols)
                    print(chart_name, cols, cols[chart_name], first_col)
                    worksheet.write_column(1, 0,  modified_df_P12M[cols[chart_name]][first_col])

                chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

                
                
        column_indices = [0, 2, 3]
        for tablenum in range(len(tables)):
            
            table = tables[tablenum].table
            table_name = tables[tablenum].name
            # if table_name != 'Table 11':
            #     is_first_row = True  # Flag to track the first row
                
            #     num_rows_to_remove = len(table.rows) - len(modified_df_P12M[tabl[table_name]])-1  # Specify the number of rows to remove from the end
            if table_name in tabl and tabl[table_name] in modified_df_P12M:
                is_first_row = True 
                num_rows_to_remove = len(table.rows) - len(modified_df_P12M[tabl[table_name]]) - 1        
                # Remove rows from the end of the table
                for _ in range(num_rows_to_remove):
                    if len(table.rows) > 1 :  # Skip removing the first row if there is more than one row
                        row = table.rows[1]
                        remove_row(table, row)
                
                table_height = Inches(3.96)  # Specify the desired table height
                total_row_height = table_height - table.rows[0].height
                num_rows = len(table.rows)-1  # Exclude the first row
                if num_rows > 0:
                    cell_height = total_row_height / num_rows
                    for row in range(1, table.rows.__len__()):
                        table.rows[row].height = int(cell_height)
                        
                        
                for i, row in enumerate(table.rows):
    
                    for j, cell in enumerate(row.cells):
                        # if is_first_row:
                        if i ==0:
                            # cell.text_frame.paragraphs[0].font.size = Pt(8)
                            if j ==3:
                                cell.text = f"Av Price /Vol ({currency.strip()})"
                            # is_first_row = False
                            if j == 0 :
                                # if list_tables[tablenum-1] == 'Seasonal And Novelties':
                                #     cell.text = "Seasonal & Novelties"
                                # elif "Seasonal And Novelties" in list_tables[tablenum-1]:
                                #     cell.text = list_tables[tablenum-1].replace("Seasonal And Novelties", "S & N")
                                    
                                # else:
                                cell.text = list_tables[tablenum-1]
                                
                            cell.text_frame.paragraphs[0].font.size = Pt(8)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                            #break

                        if j in column_indices and i !=0:
                            modified_manuf_P12M_chart = {}
                            column_index = column_indices.index(j)
                            modified_manuf_P12M_chart[tabl[table_name]] = modified_df_P12M[tabl[table_name]].copy()
                            modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M']] = modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M']].astype(float)*100
                            modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M','Av Price/KG P12M']] = round(modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M', 'Av Price/KG P12M']].astype(float),decimals) 
                            cell.text = str(modified_manuf_P12M_chart[tabl[table_name]][[f'{first_col}', 'Share DYA P12M', 'Av Price/KG P12M']].iloc[i - 1, column_index])
                            cell.fill.solid()
                            cell.fill.fore_color.rgb = RGBColor(255, 255, 255)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                            cell.text_frame.paragraphs[0].font.size = Pt(8) if isinstance(i, int) else Pt(9)
                            if j == 0:
                                cell.text_frame.paragraphs[0].font.size = Pt(9)
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            
                                
                            if j == 3:
                                # if sign == "Before":
                                #     cell.text=str(f'{currency}')+str(round(float(cell.text),decimals))
                                # else:
                                #     cell.text=str(round(float(cell.text),decimals))+str(f'{currency}')

                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                
    
                            if j == 2:
                                # cell.text=str(round(float(cell.text),1))
                                val = round(float(cell.text), 1)
               
           
                                if float(cell.text) <=-0.2:
                                    # cell.text = cell.text+'%'
                                    cell.text = f"{val}%"
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192,0,0)
   
                                elif float(cell.text) >=0.2:
                                    # cell.text = cell.text+'%'
                                    cell.text = f"+{val}%"
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
   
                                else:
                                    # if float(cell.text)==0:
                                    #    cell.text = '0'+'%'
                                    # else:
                                    #    cell.text = cell.text+'%'
                                    cell.text = "0%" if val == 0 else f"{val}%"
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
    
            else:
                pass

In [10]:
def Share_Growth_Separately_area(prs, list_duplicates, modified_df_P12M, position = 0,slide_by = '',list_tables=[],first_col=""):
    
    for slidenum in range(len(list_duplicates)):
        shapes = prs.slides[slidenum + position].shapes
        charts = []
        tables = []

        shapes = prs.slides[slidenum + position].shapes

        titleNumber = get_shape_number(shapes, "Share and Growth By Manufacturer | By Sector | National | P12M") #9
        headerNumber = get_shape_number(shapes, "Segment Leadership Analysis (Replace with So What)") #10
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022") #8

        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            # shapes[9].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
            # shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = shapes[titleNumber].text_frame.paragraphs[0].runs[0].text.replace('Category', key.split(' | ')[0]).replace('Retailer', key.split(' | ')[1])
            # shapes[9].text_frame.paragraphs[0].runs[0].text = ""
            shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = f'Share and Growth By {slide_by} | ' + list_duplicates[slidenum] +' | P12M'

            # shapes[titleNumber].text_frame.paragraphs[1].font.name = 'Nexa Bold (Headings)'
            shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'


        print(list_duplicates[slidenum])
        try:
            names = {7: 'T1',
                3 :'T3', 
                31 :'T2'}
            char_names = {12:'T1-C1',
            27:'T3-C1',
                38:'T2-C1'}
            cols = {
            'T1-C1':f'{list_duplicates[slidenum]} | {list_tables[0]}',
            'T3-C1':f'{list_duplicates[slidenum]} | {list_tables[1]}',
            'T2-C1':f'{list_duplicates[slidenum]} | {list_tables[2]}' 
        }
            tabl = {
            'T1':f'{list_duplicates[slidenum]} | {list_tables[0]}',
            'T3':f'{list_duplicates[slidenum]} | {list_tables[1]}',
            'T2':f'{list_duplicates[slidenum]} | {list_tables[2]}' 
                }

        except:
            try:
                names = {7: 'T1',
                    31 :'T2'
                    }
                char_names = {12:'T1-C1',
                38:'T2-C1'}
                cols = {
                'T1-C1':f'{list_duplicates[slidenum]} | {list_tables[0]}',
                'T2-C1':f'{list_duplicates[slidenum]} | {list_tables[1]}' 
            }
                tabl = {
                'T1':f'{list_duplicates[slidenum]} | {list_tables[0]}',
                'T2':f'{list_duplicates[slidenum]} | {list_tables[1]}' 
                    }

            except:
                names = {7: 'T1'
                    }
                char_names = {12:'T1-C1'}
                cols = {
                'T1-C1':f'{list_duplicates[slidenum]} | {list_tables[0]}'
            }
                tabl = {
                'T1':f'{list_duplicates[slidenum]} | {list_tables[0]}'
                    }
           
        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                if shape_id in char_names:
                    shape.name = char_names[shape_id]
                charts.append(shape)
            if shape.has_table:
                # print(shape.shape_id, shape.name)
                shape_id = shape.shape_id
                if shape_id in names:
                    shape.name = names[shape_id]
    
                tables.append(shape)

        for chartnum in range(len(charts)):
            chart = charts[chartnum].chart
           
            chart_name = charts[chartnum].name
            chart_type = chart.chart_type
            chart_data = CategoryChartData()
            chart_data.categories = ['']
    
            if chart_name in cols.keys():
                chart_data.add_series(cols[chart_name], modified_df_P12M[cols[chart_name]]['Value Share P12M'])
                chart.replace_data(chart_data)
            xlsx_file=BytesIO()

            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 0,first_col)

                worksheet.write_column(1, 0,  modified_df_P12M[cols[chart_name]][first_col])

            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

                
                
        column_indices = [0, 2, 3]

        for tablenum in range(len(tables)):
            table = tables[tablenum].table
            table_name = tables[tablenum].name
            # if table_name != 'Table 11':
            #     is_first_row = True  # Flag to track the first row
            #     num_rows_to_remove = len(table.rows) - len(modified_df_P12M[tabl[table_name]])-1  # Specify the number of rows to remove from the end
            if table_name in tabl and tabl[table_name] in modified_df_P12M:
                is_first_row = True 
                num_rows_to_remove = len(table.rows) - len(modified_df_P12M[tabl[table_name]]) - 1    
                # Remove rows from the end of the table
                for _ in range(num_rows_to_remove):
                    if len(table.rows) > 1 :  # Skip removing the first row if there is more than one row
                        row = table.rows[1]
                        remove_row(table, row)
                
                table_height = Inches(3.96)  # Specify the desired table height
                total_row_height = table_height - table.rows[0].height
                num_rows = len(table.rows)-1  # Exclude the first row
                if num_rows > 0:
                    cell_height = total_row_height / num_rows
                    for row in range(1, table.rows.__len__()):
                        table.rows[row].height = int(cell_height)
                        
                        
                for i, row in enumerate(table.rows):
    
                    for j, cell in enumerate(row.cells):
                        # if is_first_row:
                        if i ==0:
                            cell.text_frame.paragraphs[0].font.size = Pt(8)
                            if j == 3:
                                cell.text = f"Avg Price /Vol ({currency.strip()})"

                            # is_first_row = False
                            if j == 0 :
                                # if list_tables[tablenum-1] == 'Seasonal And Novelties':
                                #     cell.text = "Seasonal & Novelties"
                                # elif "Seasonal And Novelties" in list_tables[tablenum-1]:
                                #     cell.text = list_tables[tablenum-1].replace("Seasonal And Novelties", "S & N")
                                    
                                # else:
                                cell.text = list_tables[tablenum-1]
                            
                            cell.text_frame.paragraphs[0].font.size = Pt(8)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                            #break

                        if j in column_indices and i !=0:
                            modified_manuf_P12M_chart = {}
                            column_index = column_indices.index(j)
                            modified_manuf_P12M_chart[tabl[table_name]] = modified_df_P12M[tabl[table_name]].copy()
                            modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M']] = modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M']].astype(float)*100
                            modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M','Av Price/KG P12M']] = round(modified_manuf_P12M_chart[tabl[table_name]][['Share DYA P12M', 'Av Price/KG P12M']].astype(float),decimals) 
                            cell.text = str(modified_manuf_P12M_chart[tabl[table_name]][[f'{first_col}', 'Share DYA P12M', 'Av Price/KG P12M']].iloc[i - 1, column_index])
                            cell.fill.solid()
                            cell.fill.fore_color.rgb = RGBColor(255, 255, 255)
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                            cell.text_frame.paragraphs[0].font.size = Pt(8) if isinstance(i, int) else Pt(9)
                            if j == 0:
                                cell.text_frame.paragraphs[0].font.size = Pt(9)
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            
                                
                            if j == 3:
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                
    
                            if j == 2:
                                # cell.text=str(round(float(cell.text),1))
                                val = round(float(cell.text), 1)
           
                                if val <=-0.2:
                                    # cell.text = cell.text+'%'
                                    cell.text = f"{val}%"
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(192,0,0)
   
                                elif val >=0.2:
                                    # cell.text = cell.text+'%'
                                    cell.text = f"+{val}%"
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0,160,151)
   
                                else:
                                    # if float(cell.text)==0:
                                    #    cell.text = '0'+'%'
                                    # else:
                                    #    cell.text = cell.text+'%'
                                    cell.text = "0%" if val == 0 else f"{val}%"
                                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
    
            else:
                pass

## Slide_8:Category_Trend

In [ ]:
def Category_Trend(prs, list_duplicates, modified_df, position = 0):
    """
    Updates the Category Trend slides with data and formatting.

    Parameters:
        prs (Presentation): PowerPoint presentation object.
        list_duplicates (list): List of duplicate names for identifying slides.
        modified_df (dict): Dictionary of modified DataFrames for each duplicate name.
        position (int, optional): Position offset for slides. Defaults to 0.
    """
    for slidenum in range(len(list_duplicates)):
        shapes = prs.slides[slidenum + position].shapes
        charts = []
        tables = []
        title = shapes.title.text
        

        titleNumber = get_shape_number(shapes, "Value Sales Vs. Avg Price Trend | Confectionary | Category Trends | National") #9
        headerNumber = get_shape_number(shapes, 'Category Trends (Replace with So What)') #10
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022") #8

       
        if titleNumber is not None:
            shapes[datasourcenum].text_frame.paragraphs[0].runs[0].text = data_source
            split_parts = list_duplicates[slidenum].split(" | ")
            for brand in client_brands:
                if brand in list_duplicates[slidenum]:
                    shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = f'Value Sales Vs. Avg Price Trend | {categories[0]} | {split_parts[0]} | Category Trends | {split_parts[1]}'
                    break
            else:
                shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = f'Value Sales Vs. Avg Price Trend | {split_parts[0]} | Category Trends | {split_parts[1]}'

            shapes[titleNumber].text_frame.paragraphs[0].font.bold = True
        
        # Collect charts from the slide
        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                charts.append(shape)
                
         # Update each chart with data
        for chartnum in range(len(charts)):
            chart = charts[chartnum].chart
            chart_name = charts[chartnum].name
            chart_type = chart.chart_type
            
            # Prepare chart data
            chart_data = ChartData()
            chart_data.categories = modified_df[list_duplicates[slidenum]].iloc[:,0]
            chart_data.add_series('Value Sales', modified_df[list_duplicates[slidenum]]['Value Sales'].astype(int))
            chart_data.add_series('Av Price/KG', modified_df[list_duplicates[slidenum]]['Av Price/KG'])
            chart.replace_data(chart_data)
    
            for series in chart.series:
                for idx, point in enumerate(series.points):                      
                    if idx in [0, 13,len(series.points)-1]:
                        data_label = point.data_label
                        data_label.has_text_frame = True
                        if series.name == 'Av Price/KG': 
                            data_label.position = XL_LABEL_POSITION.ABOVE
                            if sign == 'Before':
                                data_label.text_frame.text = f"{currency}""{}".format(round(series.values[idx],decimals)) + '0' if len(str(round(series.values[idx], decimals)).split('.')[1])==1 else f"{currency}""{}".format(round(series.values[idx],decimals))
                            else:
                                data_label.text_frame.text = "{}".format(round(series.values[idx],decimals)) + '0' +f"{currency}" if len(str(round(series.values[idx], decimals)).split('.')[1])==1 else "{}".format(round(series.values[idx],decimals)) + f"{currency}"
                                
                                
                            if idx == len(series.points)-1:
                                if sign == 'Before':
                                    data_label.text_frame.text = f"Av Price/Vol \n {currency}""{}".format(round(series.values[idx],decimals)) +'0' if len(str(round(series.values[idx], decimals)).split('.')[1])==1 else f"Av Price/Vol \n {currency}""{}".format(round(series.values[idx],decimals))
                                else:
                                    data_label.text_frame.text = f"Av Price/Vol \n""{}".format(round(series.values[idx],decimals)) +'0' +  f"{currency}" if len(str(round(series.values[idx], decimals)).split('.')[1])==1 else f"Av Price/Vol \n""{}".format(round(series.values[idx],decimals)) + f"{currency}"
                                    
                        else:
                            data_label.position = XL_LABEL_POSITION.BELOW
                            data_label.text_frame.text = "{:,}".format(round(series.values[idx]))
                            
                            if idx == len(series.points)-1:
                                data_label.text_frame.text = "Value Sales \n {:,}".format(round(series.values[idx]))


## Slide_9:Share_Evolution

In [ ]:
def Share_Evolution(prs, list_duplicates, modified_df, position = 0,colname="Top Brands"):
    """
    Updates the Share Evolution slides with data and formatting.

    Parameters:
        prs (Presentation): PowerPoint presentation object.
        list_duplicates (list): List of duplicate names for identifying slides.
        modified_df (dict): Dictionary of modified DataFrames for each duplicate name.
        position (int, optional): Position offset for slides. Defaults to 0.
        colname (str, optional): Column name identifier. Defaults to "Top Brands".
    """
    for slidenum in range(len(list_duplicates)):
        shapes = prs.slides[slidenum + position].shapes
        charts = []
        tables = []
        title = shapes.title.text
        
        titleNumber = get_shape_number(shapes, "Share Evolution index analysis | By Brands | Choc Bars | Convenience | Past 3 Years") #9
        headerNumber = get_shape_number(shapes, 'Share Evolution index analysis (Replace with So What)') #10
        datasourcenum = get_shape_number(shapes, " DATA SOURCE: Trade Panel/Retailer Data | Ending Dec 2022 ") #8

        if titleNumber is not None:
            shapes[datasourcenum].text_frame.paragraphs[0].runs[0].text = data_source
            if colname==f'{BrandOrTopB}':
                shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = 'Share Evolution index analysis | By Brands | '+ list_duplicates[slidenum]
            else:
                shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = f'Share Evolution index analysis | By {ManufOrTopC} | '+ list_duplicates[slidenum]

        # Find charts and tables
        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id 
                charts.append(shape)
        
            if shape.has_table:
                shape_id = shape.shape_id
                tables.append(shape)
        
        cols= {'C1':'Price Evolution', 
              'C2':'Share Evolution'}
        
        share_evol_min = modified_df[list_duplicates[slidenum]]['Share Evolution'].min()
        share_evol_max = modified_df[list_duplicates[slidenum]]['Share Evolution'].max()

        price_evol_min = modified_df[list_duplicates[slidenum]]['Price Evolution'].min()
        price_evol_max = modified_df[list_duplicates[slidenum]]['Price Evolution'].max()
        
        for chartnum in range(len(charts)):
            chart = charts[chartnum].chart
            chart_name = charts[chartnum].name
            chart_type = chart.chart_type
            chart_data = CategoryChartData()
            chart_data.categories = sorted(modified_df[list_duplicates[slidenum]]['Year'].unique().tolist())
            
            if chart_name == 'C2':
                if colname==f"{BrandOrTopB}":
                    chart_data.add_series("Total", modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f"{BrandOrTopB}"] == 'Total']['Share Evolution'])
                    for i in modified_df[list_duplicates[slidenum]][f"{BrandOrTopB}"].unique():
                        if i != 'Total':
                            chart_data.add_series(str(i), modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f"{BrandOrTopB}"] == i]['Share Evolution'])
               
                else:
                    chart_data.add_series("Total", modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f'{ManufOrTopC}'] == 'Total']['Share Evolution'])
                    for i in modified_df[list_duplicates[slidenum]][f'{ManufOrTopC}'].unique():
                        if i != 'Total':
                            chart_data.add_series(str(i), modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f'{ManufOrTopC}'] == i]['Share Evolution'])
                axis_min = modified_df[list_duplicates[slidenum]]['Share Evolution'].min()
                axis_max = modified_df[list_duplicates[slidenum]]['Share Evolution'].max()  

            else:
                if colname==f"{BrandOrTopB}":
                    chart_data.add_series("Total", modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f"{BrandOrTopB}"] == 'Total']['Price Evolution'])
                    
                    for i in modified_df[list_duplicates[slidenum]][f"{BrandOrTopB}"].unique():
                        if i != 'Total':
                            chart_data.add_series(str(i), modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f"{BrandOrTopB}"] == i]['Price Evolution'])
                else:
                    chart_data.add_series("Total", modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f'{ManufOrTopC}'] == 'Total']['Price Evolution'])
                    
                    for i in modified_df[list_duplicates[slidenum]][f'{ManufOrTopC}'].unique():
                        if i != 'Total':
                            chart_data.add_series(str(i), modified_df[list_duplicates[slidenum]][modified_df[list_duplicates[slidenum]][f'{ManufOrTopC}'] == i]['Price Evolution'])        
                axis_min = modified_df[list_duplicates[slidenum]]['Price Evolution'].min()
                axis_max = modified_df[list_duplicates[slidenum]]['Price Evolution'].max()  
            
            # Set chart titles
            if colname==f'{BrandOrTopB}':
                if chart_name == 'C1':  # Price Evolution Chart
                    chart.chart_title.text_frame.text = "Change in average price/Vol by Brand, index"
                elif chart_name == 'C2':  # Share Evolution Chart
                    chart.chart_title.text_frame.text = "Change in average share by Brand, index"
            else:
                if chart_name == 'C1':  # Price Evolution Chart
                    chart.chart_title.text_frame.text = f"Change in average price/Vol by {ManufOrTopC}, index"
                elif chart_name == 'C2':  # Share Evolution Chart
                    chart.chart_title.text_frame.text = f"Change in average share by {ManufOrTopC}, index"
            
            chart.chart_title.text_frame.paragraphs[0].font.bold = False  # Ensure the text is not bold
            chart.chart_title.text_frame.paragraphs[0].font.name = "Nexa (Headings)"
            chart.chart_title.text_frame.paragraphs[0].font.size = Pt(8)
            chart.chart_title.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            chart.replace_data(chart_data) 
            
            value_axis = chart.value_axis
            value_axis.minimum_scale = 0.8 * axis_min
            value_axis.maximum_scale = 1.2 * axis_max
            
            # Format axis numbers to 0 decimal places
            value_axis.tick_labels.number_format = '0'    

            # Define color palettes
            client_colors = [RGBColor(0, 80, 75), RGBColor(0, 108, 109), RGBColor(0, 160, 151), RGBColor(126, 202, 196), RGBColor(153, 199, 197), RGBColor(178, 223, 220)]
            gray_colors = [RGBColor(174, 171, 171), RGBColor(166, 166, 166), RGBColor(155, 152, 152), RGBColor(127, 127, 127), RGBColor(87, 85, 85)]

            # Format series with updated coloring logic
            client_series_count = 0
            non_client_series_count = 0
            
            for i, series in enumerate(chart.series):
                for point in series.points:
                    data_label = point.data_label
                    # Set the label position (adjust as needed)
                    data_label.position = XL_LABEL_POSITION.RIGHT
                    data_label.font.size=Pt(8)
                    #data_label.ShowSeriesName = True
                    
                # Total series formatting
                if series.name == "Total":
                    series.format.line.dash_style = MSO_LINE_DASH_STYLE.ROUND_DOT
                    series.format.line.fill.solid()
                    series.format.line.fill.fore_color.rgb = RGBColor(0, 0, 0)
                    series.format.line.width = Pt(2)

                # Client brand formatting (when in brand mode)
                elif colname == f'{BrandOrTopB}' and series.name in client_brands:
                    series.format.line.dash_style = MSO_LINE_DASH_STYLE.SOLID
                    series.format.line.fill.solid()
                    series.format.line.fill.fore_color.rgb = client_colors[client_series_count % len(client_colors)]
                    series.format.line.width = Pt(1.5)
                    client_series_count += 1

                # Client manufacturer formatting (when in manufacturer mode)
                elif colname != f'{BrandOrTopB}' and series.name in client_manuf:
                    series.format.line.dash_style = MSO_LINE_DASH_STYLE.SOLID
                    series.format.line.fill.solid()
                    series.format.line.fill.fore_color.rgb = client_colors[client_series_count % len(client_colors)]
                    series.format.line.width = Pt(1.5)
                    client_series_count += 1

                # Other series formatting (non-client brands/manufacturers)
                else:
                    series.format.line.dash_style = MSO_LINE_DASH_STYLE.SOLID
                    series.format.line.fill.solid()
                    series.format.line.fill.fore_color.rgb = gray_colors[non_client_series_count % len(gray_colors)]
                    series.format.line.width = Pt(1.5)
                    non_client_series_count += 1

## Slide_10:CategoryOverview

In [ ]:
def Column_Chart_Fill_Overview(chart, scope):
    """
    Fills the columns in the chart with specific colors based on the scope.

    Parameters:
        chart (Chart): The chart object to be updated.
        scope (list): List of series names that should be highlighted with client colors.
    """
    
    client_colors = [RGBColor(0, 160, 151), RGBColor(126,202,196),RGBColor(178,223,220),RGBColor(0, 108, 109),RGBColor(0, 80, 75)]
    gray_colors = [RGBColor(217,217,217), RGBColor(191,191,191), RGBColor(166,166,166), RGBColor(155,152,152), RGBColor(127,127,127)]
    
    for i, series in enumerate(chart.series):
        if series.name in scope:
            series.format.fill.solid()
            series.format.fill.fore_color.rgb = client_colors[i if i<len(client_colors) else -1]
        else:
            series.format.fill.solid()
            series.format.fill.fore_color.rgb = gray_colors[i if i<len(gray_colors) else -1]

In [ ]:
def CategoryOverview(prs,dfList,col='MonthYear',position=0,scope = [], slide_by = ''):
    """
    Updates the Category Overview slides with data and formatting.

    Parameters:
        prs (Presentation): PowerPoint presentation object.
        dfList (list): List of tuples containing DataFrames for tables and charts, and market info.
        col (str, optional): Column name to use for categories. Defaults to 'MonthYear'.
        position (int, optional): Position offset for slides. Defaults to 0.
        scope (list, optional): List of series names to highlight in the chart. Defaults to [].
        slide_by (str, optional): Column name to distinguish series in the chart. Defaults to ''.
    """
    for slidenum in range(len(dfList)):
        dfTable=dfList[slidenum][0]
        dfChart=dfList[slidenum][1]
        market=dfList[slidenum][2]
        
        shapes = prs.slides[slidenum+position].shapes
        charts = []
        tables = []
        title = shapes.title.text

        titleNumber = get_shape_number(shapes, "Sales trend on Value vs. Volume vs. Price | Total Confectionery | National | P12M ") #9
        headerNumber = get_shape_number(shapes, 'Category Overview (Replace with So What)') #10
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: | Trade Panel/Retailer Data | Ending May 2023") #8
        value_sales_currency = get_shape_number(shapes, "Value Sales (Kr M)")
       
        if titleNumber is not None:
            shapes[datasourcenum].text_frame.paragraphs[0].runs[0].text = data_source
            shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = (f'Sales trend on Value vs. Volume vs. Price | '+market+' | P12M ').replace("  ", " ")
            shapes[value_sales_currency].text_frame.paragraphs[0].runs[0].text = f"Value Sales ({currency} M)"
        
        
        
        # shapes[4].text=data_source
        # shapes[5].text=(f'Sales trend on Value vs. Volume vs. Price | '+market+' | P12M ').replace("  ", " ")
        # shapes[5].text_frame.paragraphs[0].runs[0].font.bold = True


        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                charts.append(shape)
            if shape.has_table:
                shape_id = shape.shape_id
                tables.append(shape)
        table=tables[0].table
        itr=0
        for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                    if (i==0) & (j ==7):
                        cell.text='Pricing impact'
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(9)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(87,85,85)
                        paragraph = cell.text_frame.add_paragraph()
                        paragraph.alignment=PP_ALIGN.CENTER
                        run1 = paragraph.add_run()
                        run1.text = "on growth\n\n"
                        run1.font.name = 'Nexa Bold'
                        run1.font.size = Pt(9)  # Set the font size for this part of the text
                        run1.font.color.rgb = RGBColor(87,85,85)  # Set the font size for this part of the text
                        run1.font.bold = False  # Set the font size for this part of the text
                        run2 = paragraph.add_run()
                        run2.text = dfTable['Price Impact'][0]
                        run2.font.size = Pt(9)  #
                        run2.font.color.rgb = RGBColor(0,108,109)  # Set the font size for this part of the text
                        run2.font.name = 'Nexa Bold'
                    if i >0:
                        if (j>=1) & (cell.text!=''):
                            cell.text=str(dfTable.iloc[0,itr])
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].runs[0].font.size = Pt(9)
                            itr+=1

                    if (i==1) & (j==0):
                        cell.text = f"Value Sales ({currency} M)"  
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].runs[0].font.size = Pt(9)
                        cell.text_frame.paragraphs[0].runs[0].font.bold = True


        # CHART #
        
        chart = charts[0].chart
        chartTable=dfChart[~dfChart[col].str.contains('Total')]

        chart_data = CategoryChartData()
        chart_data.categories = chartTable[col].unique()[-36:]

        seriesLength=len(chart.series)
        toBeRemove = seriesLength-chartTable[f"{slide_by}"].nunique()-1
        for i in range(toBeRemove):
            chart.series._element[1].remove(chart.series._element[1][3])

        for sec in chartTable[f'{slide_by}'].unique():
            chart_data.add_series(sec, list(chartTable[chartTable[f'{slide_by}']==sec]['Value Sales'][-36:]/1000000))
            
        chart_data.add_series('Total',list(dfChart[(dfChart[col].str.contains('Total'))&(~dfChart[col].str.contains('Grand'))]['Value Sales'][-36:]/1000000))
        chart.replace_data(chart_data)
        Column_Chart_Fill_Overview(chart,scope)


## Slide_12: Category Evolution Analysis

In [ ]:
colorRule = {90:RGBColor(192, 0, 0),98:RGBColor(255, 128, 128),102:RGBColor(178, 223, 220),110:RGBColor(0, 160, 151),0:RGBColor(87, 85, 85)}


In [ ]:
def categoryEvolution(prs,dictionaryOfClientDf,dictionaryOfTotalDf,TotalCat,TotalSec,brandOrManufacturerList,col='Sector',position = 0):
    for slidenum,key in enumerate(dictionaryOfClientDf.keys()):
#         print(key)
        tablColDic = {2:'Value Sales IYA',3:'Volume Sales IYA',5:'IYA Price/KG',7:'Value Sales IYA',8:'Volume Sales IYA',10:'IYA Price/KG'}
        shapes = prs.slides[slidenum+position].shapes
        tables, charts = createTableAndChart(shapes)
        title = shapes.title.text
#         print( list(set(key.split(' | ')).intersection(set(brandOrManufacturerList))),set(key.split(' | ')),brandOrManufacturerList)
        brandOrManufacturer = list(set(key.split(' | ')).intersection(set(brandOrManufacturerList)))[0]
        dfClient = dictionaryOfClientDf[key].reset_index(drop=True)
        keyTotal = key.replace(' | '+brandOrManufacturer,'')#.replace(' | ','')
        
        header_num = get_shape_number(shapes,"Category Evolution Analysis (Replace with So What)")
        title_num = get_shape_number(shapes,"Sales and Growth By Sector | National | P12M")
        # print(title_num)
        data_source_num = get_shape_number(shapes,"DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022")
        # print(data_source_num)
        shapes[data_source_num].text = data_source
        shapes[title_num].text = shapes[title_num].text.replace('Sector',col).replace('National',keyTotal.split(' | ')[1])
        shapes[title_num].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[title_num].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        shapes[header_num].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[header_num].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        dfTotal =  dictionaryOfTotalDf[keyTotal].reset_index(drop=True)
#         col = 'Sector'
        missingSector=list(set(dfTotal[f"{col}"].unique())-set(dfClient[f"{col}"].unique()))
        missingSector=pd.DataFrame({f"{col}":missingSector})
        dfSectorTotal=pd.concat([dfClient,missingSector]).replace(np.nan,0).reset_index(drop=True)#.sort_values(by='Size' , ascending = False)
        dfTotal = pd.concat([TotalCat[keyTotal],dfTotal]).reset_index(drop=True)
       

        dfSectorTotal = pd.concat([TotalSec[key],dfSectorTotal]).reset_index(drop=True)
        
        dfSectorTotal = dfSectorTotal.merge(dfTotal.reset_index()[['index',col]]).sort_values(by='index').drop(columns='index').reset_index(drop=True)

        table=tables[1].table
        num_rows_to_remove = len(table.rows)-dfTotal.shape[0]-2
        table=removeRowFromTable(table,num_rows_to_remove,2)
    
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if i == 0 and j == 6:
                    cell.text = brandOrManufacturer
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85) 
                    cell.text_frame.paragraphs[0].font.bold = True

                    
                elif i in [0,1]:
                    if i == 1 and j in [1,6]:
                        cell.text = cell.text.replace('(M)',percentstr + currency )
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85) 
                        cell.text_frame.paragraphs[0].font.bold = True

                    if i == 1 and j in [4,9]:
                        # cell.text = cell.text.replace('Avg Price /Vol',f'Avg Price /Vol ({currency.strip()})')
                        cell.text = f'Avg Price /Vol ({currency.strip()})'
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85) 
                        cell.text_frame.paragraphs[0].font.bold = True
                    pass
                

                elif j == 0:
                    if i ==2:
                        cell.text = 'Total'#cat if cat!='' else 'Sector'

                    else:
                        cell.text = dfTotal[col][i-2]
                        # cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        # cell.text_frame.paragraphs[0].font.size = Pt(8)


                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)

                elif j == 4: #Total Av Price/Vol
                   
                    value =format_number(dfTotal['Av Price/KG'][i-2], use_decimals=True, decimals=2, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    value = '' if float(value.split(' ')[0])==0 else value

                    cell.text = value.replace('  ',' ')
                elif j == 9: # Av Price/Vol
                   
                    value =format_number(dfSectorTotal['Av Price/KG'][i-2], use_decimals=True, decimals=2, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    value = '' if float(value.split(' ')[0])==0 else value
                    cell.text = value.replace('  ',' ')

                if j in [2,3,4,5,7,8,9,10] and i>=2:
                    if j in [2,3,5]:
                        value =  int(round(float(dfTotal[tablColDic[j]][i-2])*100,0))
                       
                    
                    elif j in [7,8,10]:
                        value =  int(round(float(dfSectorTotal[tablColDic[j]][i-2])*100,0))
                    if j not in [4,9]:

                        colorIndex = 90 if value <= 90 else 98 if value<=98 else 102 if (value >=102 and value < 110) else 110 if value >=110 else 0
                        colorIndex = colorRule[colorIndex]
                        value = '' if value==0 else value
                        cell.text = str(value)
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85) if j in [4,9] else colorIndex  #RGBColor(178,223,220)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
        for i,df in enumerate([dfTotal,dfSectorTotal]):

            chartTotal = charts[i].chart
            chart_data = CategoryChartData()
            chart_data.categories = df[col].to_list()
            series = chart_data.add_series('Value Sales', df['Value Sales']/percent)
            chartTotal.replace_data(chart_data)
            for j, point in enumerate(chartTotal.series[0].points):
                data_label = point.data_label
                data_label.has_text_frame = True
                if df['Value Sales'][j]/percent != 0:
                    value = format_number(round(df['Value Sales'][j]/percent), use_decimals=False, decimals=0, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                else:
                    value = ''

                data_label.text_frame.text = value.replace('  ',' ').replace(',','')
    #             data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
                data_label.text_frame.paragraphs[0].font.name = 'Nexa Book (Body)'
                data_label.text_frame.paragraphs[0].font.size = Pt(8)


## Slide 13 : Share And Growth

In [ ]:
def shareGrowth(prs,dictionaryOfTotalClientDf,dictionaryOfTotalRetailer,catSecSeg,col='Region',retailType='Retailer',position = 0):
    for slidenum,key in enumerate(dictionaryOfTotalClientDf.keys()):
        shapes = prs.slides[slidenum+position].shapes
        tables, charts = createTableAndChart(shapes)
#         print(key,slidenum,len(dictionaryOfTotalClientDf))
        cat = list(set(key.split(' | ')).intersection(set(catSecSeg)))[0]
        catType = 'Category' if cat in categories else 'Sector' if cat in sectors else 'Segment' if cat in segments else 'SubSegmet' if cat in subsegments else 'SubCategory'
       
        market = list(set(key.split(' | '))-set([cat]))[0]
        dfClient = dictionaryOfTotalClientDf[key].reset_index(drop=True)
        dfTotal = dictionaryOfTotalRetailer[cat].reset_index(drop=True)
#         dfTotal.iloc[0]['Channel'],dfTotal.iloc[0]['Region'],dfTotal.iloc[0]['MArket']='Total','Total','Total'
       
        missingRegion=list(set(dfTotal[f"{col}"].unique())-set(dfClient[f"{col}"].unique()))
        missingRegion=pd.DataFrame({f"{col}":missingRegion})
        missingRegion=missingRegion.merge(dfTotal[[col]],how='inner')
        dfClient=pd.concat([dfClient,missingRegion]).replace(np.nan,0).reset_index(drop=True)
 
        dfTotal[col] = np.where(dfTotal[col].isna(), "Grand Total", dfTotal[col])
 
        dfClient[col] = np.where(dfClient[col] == 0, "Grand Total", dfClient[col])
 
        dfClient = dfClient.merge(dfTotal.reset_index()[['index',col]]).sort_values(by='index').drop(columns='index').reset_index(drop=True)
#         dfClient.iloc[0]['Channel'],dfClient.iloc[0]['Region'],dfClient.iloc[0]['MArket']='Total','Total','Total'
 
       
        title_num = get_shape_number(shapes,"Share and Growth By Retailer | Total Category | P12M")
        data_source_num = get_shape_number(shapes,"DATA SOURCE: Trade Panel/Retailer Data | Ending Sep 2022")
        shapes[data_source_num].text = data_source
        shapes[title_num].text = shapes[title_num].text.replace('Category',catType).replace('Retailer',retailType)
        shapes[title_num].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[title_num].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        shapes[title_num+1].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[title_num+1].text_frame.paragraphs[0].font.name = 'Nexa Bold'
#         print(cat,'----',market,'----',key)
        totalTable=tables[1].table
        num_rows_to_remove = len(totalTable.rows)-dfTotal.shape[0]-2
#         print(dfTotal.shape[0],num_rows_to_remove)
       
        totalTable=removeRowFromTable(totalTable,num_rows_to_remove,2)
        for i, row in enumerate(totalTable.rows):
            for j, cell in enumerate(row.cells):
#                 print('row: ',i,' Col: ',j,' Text: ',cell.text)
                if i == 0 and j == 1:
                    cell.text = cat
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
 
                elif i == 1 and j == 4:
                    cell.text = f'Av Price/Vol ({currency.strip()})'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = True
                   
                elif i in [0,1]:
                    if i==1 and j == 1:
                        cell.text = cell.text.replace('(M)',percentstr+currency)
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = True
                       
                    pass
                elif j == 0:
                    if i ==2:
                        cell.text ='Total'# cat if cat!='' else 'Sector'
                       
                    else:
                        cell.text = dfTotal[col][i-2].replace(' Total','')
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                elif j ==4:
#                     value = int(round(dfTotal['Av Price/KG'][i-2],0))
#                     value = '' if value == 0 else value
                    value =format_number(dfTotal['Av Price/KG'][i-2], use_decimals=True, decimals=2, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    value = '' if float(value.split(' ')[0])==0 else value
 
                    cell.text = value
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                   
                elif j ==5:
                    value =  int(round(float(dfTotal['IYA Price/KG'].replace(np.nan,0)[i-2])*100,0))
                       
                    colorIndex = 90 if value <= 90 else 98 if value<=98 else 102 if (value >=102 and value < 110) else 110 if value >=110 else 0
                    colorIndex = colorRule[colorIndex]
 
                    value = '' if value==0 else value
                    cell.text = str(value)
 
#                     value = '' if value==0 else value
#                     cell.text = str(value)
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = colorIndex  #RGBColor(178,223,220)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
 
               
        clientTable=tables[2].table
#         print(num_rows_to_remove,'client')
        clientTable=removeRowFromTable(clientTable,num_rows_to_remove,2)
   
        for i, row in enumerate(clientTable.rows):
            for j, cell in enumerate(row.cells):
#                 print('row: ',i,' Col: ',j,' Text: ',cell.text)
                if i == 0 and j == 0:
                    cell.text = market
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
 
                elif i == 1 and j == 3:
                    cell.text = f'Av Price/Vol ({currency.strip()})'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                    cell.text_frame.paragraphs[0].font.bold = True
 
                   
                elif i in [0,1]:
                    if i ==1 and j==0:
                        cell.text = cell.text.replace('(M)',percentstr + currency)
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                        cell.text_frame.paragraphs[0].font.bold = True
                    pass
               
                elif j ==3:
                    value =format_number(dfClient['Av Price/KG'][i-2], use_decimals=True, decimals=2, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    value = '' if float(value.split(' ')[0])==0 else value
 
                    cell.text = value
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                elif j ==4:
                    value =  int(round(float(dfClient['IYA Price/KG'].replace(np.nan,0)[i-2])*100,0))
 
                   
                    colorIndex = 90 if value <= 90 else 98 if value<=98 else 102 if (value >=102 and value < 110) else 110 if value >=110 else 0
                    colorIndex = colorRule[colorIndex]
#                     value = '' if value==0 else value
                    value = '' if value==0 else value
                   
                    cell.text = str(value)
#                     cell.text = str(value)
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = colorIndex  #RGBColor(178,223,220)
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
 
                   
                   
        total = {0:'Trade WoB %',4:'Trade WoB % DYA',2:'Value Sales'}
        client = {1:'Trade WoB %',5:'Trade WoB % DYA',3:'Value Sales'}
        for key,value in total.items():
            chartTotal = charts[key].chart
 
            chart_data = CategoryChartData()
            chart_data.categories = dfTotal[col].replace(np.nan,'Total').replace('Grand ','').to_list()
#             print(dfTotal[col].replace('Grand ','').to_list())
            series = chart_data.add_series(value, dfTotal[value.replace('Trade ','')].replace(np.nan,0))
 
 
            chartTotal.replace_data(chart_data)
            if key == 2 :
 
                for j, point in enumerate(chartTotal.series[0].points):
 
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    if round(series.values[j]/percent) !=0:
                        value = format_number(round(series.values[j]/percent), use_decimals=False, decimals=0, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    else:
                        value = ""
 
                    data_label.text_frame.text = value.replace('  ',' ').replace(',','')
        #             data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
                    data_label.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    data_label.text_frame.paragraphs[0].font.size = Pt(8)
                    data_label.position = XL_LABEL_POSITION.CENTER
       
 
 
            elif key == 4 :
                for j, point in enumerate(chartTotal.series[0].points):
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    # if series.values[j] !=0:
                    #     value = format_number((series.values[j]*100), use_decimals=True, decimals=1, use_apostrophes=False, currency_symbol="%", currency_before=False if sign.lower=='before' else False)
                    # else:
                    #     value = ""
 
                    val = round(series.values[j] * 100, 1)
                    if val != 0:
                        prefix = "+" if val > 0 else ""
                        value = f"{prefix}{val:.1f}%"
                    else:
                        value = ""
 
                    # value = format_number(series.values[j]/percent, use_decimals=True, decimals=1, use_apostrophes=False) +'%'
    #                 value = '' if value.split(' ')[0]=='0' else value
                    data_label.text_frame.text = value.replace('  ',' ').replace(',','')
        #             data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
                    data_label.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    data_label.text_frame.paragraphs[0].font.size = Pt(8)
                    data_label.position = XL_LABEL_POSITION.CENTER
            elif key == 0 :
                for j, point in enumerate(chartTotal.series[0].points):
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    if series.values[j] !=0:
                        value = format_number(series.values[j]*100, use_decimals=True, decimals=1, use_apostrophes=False, currency_symbol="%", currency_before=False if sign.lower=='before' else False)
                    else:
                        value = ""
                    # value = format_number(series.values[j]/percent, use_decimals=True, decimals=1, use_apostrophes=False) +'%'
    #                 value = '' if value.split(' ')[0]=='0' else value
                    data_label.text_frame.text = value.replace('  ',' ').replace(',','')
        #             data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
                    data_label.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    data_label.text_frame.paragraphs[0].font.size = Pt(8)
                    data_label.position = XL_LABEL_POSITION.CENTER
 
 
 
        for key,value in client.items():
            chartClient = charts[key].chart
 
            chart_data2 = CategoryChartData()
            chart_data2.categories = dfClient[col].replace(0,'Total').replace(np.nan,'Total').replace('Grand ','').to_list()
 
            series = chart_data2.add_series(value, dfClient[value.replace('Trade ','')].replace(np.nan,0))
            chartClient.replace_data(chart_data2)
            if key == 3 :
                for j, point in enumerate(chartClient.series[0].points):
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    if round(series.values[j]/percent) !=0:
                        value = format_number(round(series.values[j]/percent), use_decimals=False, decimals=0, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    elif (series.values[j]/percent <1) & (series.values[j]/percent !=0):
                        value = format_number(series.values[j]/percent, use_decimals=True, decimals=1, use_apostrophes=False, currency_symbol="", currency_before=True if sign.lower=='before' else False)
                    else:
                        value = ""
 
                    data_label.text_frame.text = value.replace('  ',' ').replace(',','')
                    data_label.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    data_label.text_frame.paragraphs[0].font.size = Pt(8)
                    data_label.position = XL_LABEL_POSITION.CENTER
 
            elif key == 5 :
                for j, point in enumerate(chartClient.series[0].points):
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    # if series.values[j] !=0:
                    #     value = format_number((series.values[j]*100), use_decimals=True, decimals=1, use_apostrophes=False, currency_symbol="%", currency_before=False if sign.lower=='before' else False)
                    # else:
                    #     value = ""
 
                    val = round(series.values[j] * 100, 1)
                    if val != 0:
                        prefix = "+" if val > 0 else ""
                        value = f"{prefix}{val:.1f}%"
                    else:
                        value = ""
                    # value = format_number(series.values[j]/percent, use_decimals=True, decimals=1, use_apostrophes=False) +'%'
    #                 value = '' if value.split(' ')[0]=='0' else value
                    data_label.text_frame.text = value.replace('  ',' ').replace(',','')
        #             data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
                    data_label.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    data_label.text_frame.paragraphs[0].font.size = Pt(8)
                    data_label.position = XL_LABEL_POSITION.CENTER
            elif key == 1 :
                for j, point in enumerate(chartClient.series[0].points):
                    data_label = point.data_label
                    data_label.has_text_frame = True
                    if series.values[j] !=0:
                        value = format_number(series.values[j]*100, use_decimals=True, decimals=1, use_apostrophes=False, currency_symbol="%", currency_before=False if sign.lower=='before' else False)
                    else:
                        value = ""
                    # value = format_number(series.values[j]/percent, use_decimals=True, decimals=1, use_apostrophes=False) +'%'
    #                 value = '' if value.split(' ')[0]=='0' else value
                    data_label.text_frame.text = value.replace('  ',' ').replace(',','')
        #             data_label.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
                    data_label.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    data_label.text_frame.paragraphs[0].font.size = Pt(8)
                    data_label.position = XL_LABEL_POSITION.CENTER
 

## SLIDE 14 : Momentum Analysis

In [ ]:
def Momentum_Analysis(prs, list_duplicates, modified_df, position = 0):
    for key, slidenum in zip(modified_df, range(len(list_duplicates))):
        
        shapes = prs.slides[slidenum + position].shapes

        titleNumber = get_shape_number(shapes, "ERROR") #9
        headerNumber = get_shape_number(shapes, 'Momentum Analysis (Replace with So What)') #10
        datasourcenum = get_shape_number(shapes, "DATA SOURCE: Trade Panel/Retailer Data | Ending Apr 2024") #8

        df = modified_df[key].copy()
        df =df.nlargest(30, "Value Sales P12M")
        df =df.replace(np.nan, 0)

        df['index'] = range(1, len(df) + 1)

        # print(df[key].columns)
        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            # shapes[9].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
            # shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = shapes[titleNumber].text_frame.paragraphs[0].runs[0].text.replace('Category', key.split(' | ')[0]).replace('Retailer', key.split(' | ')[1])
            # shapes[9].text_frame.paragraphs[0].runs[0].text = ""
            shapes[titleNumber].text_frame.paragraphs[0].runs[0].text = f"Momentum Analysis | P12M vs. P3M Share Difference vs. YA | {key.split(' | ')[0]} | {key.split(' | ')[1]} | By Brand x Market\nBubble Size: P12M Value Sales"

            # shapes[titleNumber].text_frame.paragraphs[1].font.name = 'Nexa Bold (Headings)'
            shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'


        # Create and retrieve table and chart shapes
        tables, charts = createTableAndChart(shapes)
        chart = charts[0].chart
        table = tables[0].table

        # Adjust the number of rows in the table based on the DataFrame
        num_rows_to_remove = len(table.rows) - df.shape[0]
        #table_height = 3.85
        table = removeRowFromTable(table, num_rows_to_remove, rowToExclude=0)

        # Populate the table with product data
        for row_number, row in enumerate(table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if df.shape[0] <20:
                    if column_num == 0 and row_number < df.shape[0]:
                        value = row_number + 1
                        cell.text = str(value)
                        set_cell_font(cell, 'Nexa Book (Body)', 9)


                    if column_num == 1 and row_number < df.shape[0]:
                        value = df['Labels'].to_list()[row_number]
                        cell.text = str(value)
                        set_cell_font(cell, 'Nexa Book (Body)', 9)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                else:
                    
                    if column_num == 0 and row_number < df.shape[0]:
                        value = row_number + 1
                        cell.text = str(value)
                        set_cell_font(cell, 'Nexa Book (Body)', 6)


                    if column_num == 1 and row_number < df.shape[0]:
                        value = df['Labels'].to_list()[row_number]
                        cell.text = str(value)
                        set_cell_font(cell, 'Nexa Book (Body)', 6)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT


                cell.fill.solid()

                if (df['Value Share DYA P3M'].to_list()[row_number]>0) &(df['Value Share DYA P12M'].to_list()[row_number]>0): 
                    cell.fill.fore_color.rgb = RGBColor(203, 234, 231) # Green Color
                elif (df['Value Share DYA P3M'].to_list()[row_number]<0) &(df['Value Share DYA P12M'].to_list()[row_number]<0):
                    cell.fill.fore_color.rgb = RGBColor(255, 191, 191) # Red Color
                else:
                    cell.fill.fore_color.rgb = RGBColor(242,242,242) # Gray Color
        table.height = Inches(3.93)
                
        # Prepare bubble chart data
        chart_data = BubbleChartData()
        x_values = df['Value Share DYA P12M'].tolist()
        y_values = df['Value Share DYA P3M'].tolist()

        bubble_sizes = df['Value Sales P12M'].tolist()
        chart_data.categories = [c for c in list(df['index'])]
        series_1 = chart_data.add_series('P3M Value Share')

        # Add data points to the bubble chart
        for i in range(len(x_values)):
            series_1.add_data_point(x_values[i], y_values[i], bubble_sizes[i])
        chart.replace_data(chart_data)


        xlsx_file = BytesIO()

        # Set data labels for the chart points
        for i, point in enumerate(chart.series[0].points):
            data_label = point.data_label
            data_label.has_text_frame = True
        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 3, "Column1")
            worksheet.write(0, 4, "Labels")
            worksheet.write_column(1,4, df['Labels'].to_list(), None)
            # Write sequential numbers as labels starting from the first point in the series
            Col1 = [str(i + 1) for i, _ in enumerate(chart.series[0].points)]  # Create a list of labels as strings
            worksheet.write_column(1, 3, Col1)  # Write the list of labels to the worksheet column
            

        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
        data_label.text_frame.text = str(i + 1)
        data_label.position = XL_LABEL_POSITION.CENTER

        value_axis = chart.value_axis
        value_axis.tick_labels.number_format = '#,##0.0%'
        vertical_cross_value = df['Value Share DYA P3M'].mean()
        value_axis.crosses = XL_AXIS_CROSSES.CUSTOM
        value_axis.crosses_at = vertical_cross_value


        category_axis = chart.category_axis
        category_axis.tick_labels.number_format = '#,##0.0%'

        horizontal_cross_value = df['Value Share DYA P12M'].mean()
        category_axis.crosses = XL_AXIS_CROSSES.CUSTOM
        category_axis.crosses_at = horizontal_cross_value


## SLIDE 14 : RPVM

In [ ]:

def get_shape_number_win32com(slide, text):

    for shape_index in range(1, slide.Shapes.Count+1):  # Shape indices start from 1 in win32com
        shape = slide.Shapes(shape_index)
        if shape.HasTextFrame:  # Check if the shape has a text frame
            if shape.TextFrame.HasText:  # Check if the text frame has text
                if shape.TextFrame.TextRange.Text == text:  # Compare text
                    return shape_index  # Return the 1-based index of the shape if text matches
    return None  # Return None if the shape with the given text is not found


def RGB(red, green, blue):
    return red + (green * 256) + (blue * 256 * 256)


In [ ]:
import os
import pickle
import gc
from win32com.client import Dispatch, constants
from win32com.client import gencache

def replace_chart_title(chart, replace_with):
    try:
        # Ensure that the chart has a title
        if chart.HasTitle:
            title_text = chart.ChartTitle.Text

            if title_text:  # Check if title is not empty
                pos = title_text.find(title_text)
                if pos != -1:
                    # Replace part of the chart title with the new text
                    chart.ChartTitle.Text = title_text[:pos] + replace_with + title_text[pos + len(title_text):]
                else:
                    print(f"Text '{title_text}' not found in chart title.")
            else:
                print("Chart title is empty.")
        else:
            print("No chart title.")
    except Exception as e:
        print(f"Error accessing or modifying chart title: {e}")


# Save progress function
def save_progress(slidenum):
    with open('progress.pkl2', 'wb') as f:
        pickle.dump(slidenum, f)

# Load progress function
def load_progress():
    if os.path.exists('progress.pkl2'):
        with open('progress.pkl2', 'rb') as f:
            return pickle.load(f)
    return 0  # Default to 0 if no progress file exists

# Function to quit Excel safely
def quit_excel(excel_app):
    if 'excel_app' in locals() and excel_app:
        excel_app.Quit()  # Quit Excel only if it was initialized

# Main function to replace waterfall chart data
def RPVM(prs, list_duplicates, modified_RPVM, position=0):
    global max_c1, max_c2
    max_c1, max_c2 = 0, 0  # Initialize max values for charts
    max_axis_val = None  # Initialize max_axis_val to store the highest Y-axis value

    # Load the last processed slide number
    start_slide_num = load_progress()

    # Retry mechanism for failed slides
    max_retries = 5  # Max number of retries for each slide

    for slidenum in range(start_slide_num, len(list_duplicates)):
        retries = 0
        success = False

        while retries < max_retries and not success:
            try:
                slide = prs.Slides[slidenum + position]
                shapes = slide.shapes

                # Fetch the DataFrame for this slide
                df = modified_RPVM[list_duplicates[slidenum]].copy().fillna(0)
                c1_data = df[df[df.columns[0]].isin(client_brands + client_manuf)]
                c2_data = df[~df[df.columns[0]].isin(client_brands + client_manuf)]
                c1_data = c1_data.reset_index(drop=True)
                c2_data = c2_data.reset_index(drop=True)
                # Determine the brands
                my_brand = c1_data[df.columns[0]].unique()[0]
                compet_brand = c2_data[df.columns[0]].unique()[0]

                # Update text frames
                shapes[1].TextFrame.TextRange.Text = data_source
                shapes[2].TextFrame.TextRange.Text = (
                    f"Revenue by Price vs. Volume vs. Mix analysis | {my_brand} vs. {compet_brand} | "
                    + " | ".join(list_duplicates[slidenum].split(" | ")[:2]) + " | P12M"
                )

                # Iterate through all shapes to update charts and data
                for shape in shapes:
                    if shape.HasChart:
                        chart = shape.Chart
                        chart_data = chart.ChartData

                        # Safely handle Excel activation
                        try:
                            chart_data.Activate()
                            workbook = chart_data.Workbook
                            excel_app = workbook.Application
                            excel_app.Visible = False  # Keep Excel hidden
                            
                            sheet = workbook.Sheets(1)

                            if chart.name == "Chart 6":
                                sheet.Range(sheet.Cells(2, 1), sheet.Cells(len(c1_data) + 1, 2)).Value = c1_data[['Revenue Group', 'Revenue By PVM']].values.tolist()
                                replace_chart_title(chart, replace_with=my_brand)

                                y_axis = chart.Axes(2)  # Y-axis
                                y_axis.MaximumScaleIsAuto = True
                                max_c1 = y_axis.MaximumScale

                            elif chart.name == "Chart 12":
                                sheet.Range(sheet.Cells(2, 1), sheet.Cells(len(c2_data) + 1, 2)).Value = c2_data[['Revenue Group', 'Revenue By PVM']].values.tolist()
                                replace_chart_title(chart, replace_with=compet_brand)

                                y_axis = chart.Axes(2)  # Y-axis
                                y_axis.MaximumScaleIsAuto = True
                                max_c2 = y_axis.MaximumScale

                            # Update max_axis_val for both charts
                            max_axis_val = max(max_c1, max_c2)
                            workbook.Close(SaveChanges=True)

                            for shape in shapes:
                                if shape.HasChart:
                                    chart = shape.Chart
                                    if chart.name in ["Chart 6", "Chart 12"]:
                                        y_axis = chart.Axes(2)
                                        y_axis.MaximumScale = max_axis_val

                                        series = chart.SeriesCollection(1)

                                        for i, point in enumerate(series.Points()):
                                            value = (c1_data if chart.name == "Chart 6" else c2_data).loc[i, "Revenue By PVM"]#["Revenue By PVM"].iloc[i]
                                            category = (c1_data if chart.name == "Chart 6" else c2_data).loc[i, "Revenue Group"]#["Revenue Group"].iloc[i]

                                            if category in ["Price", "Volume", "Mix", "New", "Discontinued"]:
                                                if value < 0:
                                                    point.Format.Fill.ForeColor.RGB = RGB(192, 0, 0)  # Red
                                                elif value > 0:
                                                    point.Format.Fill.ForeColor.RGB = RGB(0, 160, 151)  # Green
                                                else:
                                                    point.Format.Fill.ForeColor.RGB = RGB(174, 171, 171)  # Grey
                                            else:
                                                point.Format.Fill.ForeColor.RGB = RGB(174, 171, 171)


                            # Conditional formatting for series points
                            # series = chart.SeriesCollection(1)

                            # for i, point in enumerate(series.Points()):
                            #     value = (c1_data if chart.name == "Chart 6" else c2_data).loc[i, "Revenue By PVM"]#["Revenue By PVM"].iloc[i]
                            #     category = (c1_data if chart.name == "Chart 6" else c2_data).loc[i, "Revenue Group"]#["Revenue Group"].iloc[i]

                            #     if category in ["Price", "Volume", "Mix", "New", "Discontinued"]:
                            #         if value < 0:
                            #             point.Format.Fill.ForeColor.RGB = RGB(192, 0, 0)  # Red
                            #         elif value > 0:
                            #             point.Format.Fill.ForeColor.RGB = RGB(0, 160, 151)  # Green
                            #         else:
                            #             point.Format.Fill.ForeColor.RGB = RGB(174, 171, 171)  # Grey
                            #     else:
                            #         point.Format.Fill.ForeColor.RGB = RGB(174, 171, 171)

                        except Exception as e:
                            print(f"Error updating chart data on slide {slidenum}: {e}")
                        finally:
                            quit_excel(excel_app)  # Quit Excel only if it was initialized

                prs.Application.ActiveWindow.View.GotoSlide(slidenum + 2)

                # Save progress after successful processing
                save_progress(slidenum + 1)
                success = True  # Mark as successful if no error occurs

            except Exception as e:
                retries += 1
                print(f"Error on slide {slidenum}, retry {retries}: {e}")
                if retries ==1:
                    time.sleep(5)
                elif retries == 4:
                    time.sleep(15)

                if retries >= max_retries:
                    print(f"Max retries reached for slide {slidenum}. Moving to the next slide.")
                    save_progress(slidenum)  # Save progress on error
                    break  # Exit the retry loop

        # Cleanup resources after processing all slides
        if os.path.exists('progress.pkl2'):
            os.remove('progress.pkl2')
        gc.collect()

## Some Cleaning Functions

In [ ]:
def process_RET_CHAN_POS(dataframes,regions, channels, markets):
    modified_regions = {}
    modified_channels = {}
    modified_markets = {}
    dya = {}

    for k in dataframes.keys():
        t = dataframes[k].copy()
        t=DetectHeader(t)
        t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        t['Value Share'] = t['Value Share'].astype(float)
        t = t.sort_values('Value Share', ascending=False)
        # t=t.drop(columns=['WoB %', "Relative Price", "Growth Contribution"])
        t.columns = t.columns.str.replace('Trade ', '').str.replace('Channel ','').str.strip()
        t = t.sort_values('WoB %', ascending=False)  
        t = t.drop_duplicates()
        
        level_check = set(["Region", "Channel", "Market"])
        existant_cols = list(set(t.columns)&level_check)

        decider_dic = {"Region": regions, "Channel":channels, "Market":markets}
        grand_tot = t[t[t.columns[0]] == 'Grand Total'] 
        if not grand_tot.empty:
            dya[k]=grand_tot

        if len(existant_cols) == 1:
            df = t[t[t.columns[0]].isin(decider_dic[existant_cols[0]])]
            if not df.empty:
                if df.columns[0] == "Region":
                    modified_regions[k] = df
                if df.columns[0] == "Channel":
                    modified_channels[k] = df
                if df.columns[0] == "Market":
                    modified_markets[k] = df

        elif len(existant_cols) == 2:
            levels_rank = {"Region":1, "Channel":2, "Market":3}
            existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])
            t_child = t[t[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns = [existant_cols[0]])
            t_parent = t[t[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns = [existant_cols[1]])
            t_parent[existant_cols[0]] = t_parent[existant_cols[0]].str.replace(" Total", "").str.strip()
            for df in [t_child, t_parent]:
                if not df.empty:
                    if df.columns[0] == "Region":
                        modified_regions[k] = df
                    if df.columns[0] == "Channel":
                        modified_channels[k] = df
                    if df.columns[0] == "Market":
                        modified_markets[k] = df
      
        else:
            levels_rank = {"Region":1, "Channel":2, "Market":3}
            existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])
            t_market = t[t[existant_cols[2]].isin(decider_dic[existant_cols[2]])].drop(columns = [existant_cols[0],existant_cols[1]])
            t_channel = t[t[existant_cols[1]].isin([i + " Total" for i in decider_dic[existant_cols[1]]])].drop(columns = [existant_cols[0],existant_cols[2]])
            t_channel[existant_cols[1]] = t_channel[existant_cols[1]].str.replace(" Total", "").str.strip()
            t_region = t[t[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns = [existant_cols[1],existant_cols[2]])
            t_region[existant_cols[0]] = t_region[existant_cols[0]].str.replace(" Total", "").str.strip()
            if not t_region.empty:
                modified_regions[k] = t_region
            if not t_channel.empty:
                modified_channels[k] = t_channel
            if not t_market.empty:
                modified_markets[k] = t_market
            # if not grand_tot.empty:
            #     dya[k]=grand_tot
    return modified_regions, modified_channels, modified_markets, dya
            



In [ ]:
# def process_RET_CHAN_Momentum(dataframes, regions, channels, markets, client_brands, area_by=""):
#     # Dictionary to hold all modified DataFrames in a single structure
#     modified_data = {}

#     for k, df in dataframes.items():
#         # Copy and process each DataFrame
#         t = df.copy()

#         t.iloc[15] = t.iloc[15].ffill().replace(np.nan, "") 
#         t.columns = (t.iloc[16] +" "+ t.iloc[15]).str.strip()
#         t = t.iloc[17:]
#         if not t.empty:
#             t['Top Brands'] = t['Top Brands'].ffill()


#             top_brands_totals = t[(t['Top Brands'].str.contains("Total")) & (t['Top Brands']!="Grand Total") & (~t["Top Brands"].isin([i+" Total" for i in client_brands]))]
#             competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)['Top Brands'].to_list()
#             competitors = [i.replace(" Total","") for i in competitors]

#             t_filtered = t[t['Top Brands'].isin(client_brands) |
#                                     ((t['Top Brands'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]


#             # Convert applicable columns to float
#             t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
#             t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
#             t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)


#             # Identify relevant columns for levels and filter data
#             level_check = set(["Region", "Channel", "Market"])
#             existant_cols = list(set(t_filtered.columns) & level_check)
#             decider_dic = {"Region": regions, "Channel": channels, "Market": markets}

#             key_name = f"{k} | {area_by}" if area_by else k

#             # If there is a single level (Region, Channel, or Market)
#             if len(existant_cols) == 1:
#                 filtered_df = t_filtered[t_filtered[t_filtered.columns[1]].isin(decider_dic[existant_cols[0]])]
#                 if not filtered_df.empty:

#                     # Rename the existing column to 'Retailer'
#                     filtered_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
#                     filtered_df['Labels'] = filtered_df['Top Brands'] + "/" + filtered_df[f"{area_by}"]  
#                     modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), filtered_df], ignore_index=True)
#                     modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
#                     modified_data[key_name] = modified_data[key_name].drop_duplicates()


#             # If there are two levels
#             elif len(existant_cols) == 2:
#                 levels_rank = {"Region": 1, "Channel": 2, "Market": 3}
#                 existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])

#                 child_df = t_filtered[t_filtered[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns=[existant_cols[0]])
#                 parent_df = t_filtered[t_filtered[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns=[existant_cols[1]])
#                 parent_df[existant_cols[0]] = parent_df[existant_cols[0]].str.replace(" Total", "").str.strip()

#                 # Rename columns and append to the combined DataFrame
#                 if not child_df.empty:
#                     child_df.rename(columns={existant_cols[1]: f"{area_by}"}, inplace=True)
#                     child_df['Labels'] = child_df['Top Brands'] + "/" + child_df[f"{area_by}"]  
#                     modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), child_df], ignore_index=True)
#                     modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
#                     modified_data[key_name] = modified_data[key_name].drop_duplicates()


#                 if not parent_df.empty:
#                     parent_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
#                     parent_df['Labels'] = parent_df['Top Brands'] + "/" + parent_df[f"{area_by}"]  
#                     modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), parent_df], ignore_index=True)
#                     modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
#                     modified_data[key_name] = modified_data[key_name].drop_duplicates()




#             # If there are three levels
#             elif len(existant_cols) == 3:
#                 levels_rank = {"Region": 1, "Channel": 2, "Market": 3}
#                 existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])

#                 market_df = t_filtered[t_filtered[existant_cols[2]].isin(decider_dic[existant_cols[2]])].drop(columns=[existant_cols[0], existant_cols[1]])
#                 channel_df = t_filtered[t_filtered[existant_cols[1]].isin([i + " Total" for i in decider_dic[existant_cols[1]]])].drop(columns=[existant_cols[0], existant_cols[2]])
#                 channel_df[existant_cols[1]] = channel_df[existant_cols[1]].str.replace(" Total", "").str.strip()
#                 region_df = t_filtered[t_filtered[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns=[existant_cols[1], existant_cols[2]])
#                 region_df[existant_cols[0]] = region_df[existant_cols[0]].str.replace(" Total", "").str.strip()

#                 # Rename columns and append to the combined DataFrame
#                 if not market_df.empty:
#                     market_df.rename(columns={existant_cols[2]: f"{area_by}"}, inplace=True)
#                     market_df['Labels'] = market_df['Top Brands'] + "/" + market_df[f"{area_by}"]  

#                     modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), market_df], ignore_index=True)
#                     modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
#                     modified_data[key_name] = modified_data[key_name].drop_duplicates()



#                 if not channel_df.empty:
#                     channel_df.rename(columns={existant_cols[1]: f"{area_by}"}, inplace=True)
#                     channel_df['Labels'] = channel_df['Top Brands'] + "/" + channel_df[f"{area_by}"]  
#                     modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), channel_df], ignore_index=True)
#                     modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
#                     modified_data[key_name] = modified_data[key_name].drop_duplicates()



#                 if not region_df.empty:
#                     region_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
#                     region_df['Labels'] = region_df['Top Brands'] + "/" + region_df[f"{area_by}"]  
#                     modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), region_df], ignore_index=True)
#                     modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
#                     modified_data[key_name] = modified_data[key_name].drop_duplicates()



#     return modified_data

In [ ]:
def process_RET_CHAN_Momentum(dataframes, regions, channels, markets, client_brands, area_by=""):
    # Dictionary to hold all modified DataFrames in a single structure
    modified_data = {}

    for k, df in dataframes.items():
        # Copy and process each DataFrame
        t = df.copy()

        t.iloc[15] = t.iloc[15].ffill().replace(np.nan, "") 
        t.columns = (t.iloc[16] +" "+ t.iloc[15]).str.strip()
        t = t.iloc[17:]
        if not t.empty:
            t['Top Brands'] = t['Top Brands'].ffill()


            top_brands_totals = t[(t['Top Brands'].str.contains("Total")) & (t['Top Brands']!="Grand Total") & (~t["Top Brands"].isin([i+" Total" for i in client_brands]))]
            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)['Top Brands'].to_list()
            competitors = [i.replace(" Total","") for i in competitors]

            t_filtered = t[t['Top Brands'].isin(client_brands) |
                                    ((t['Top Brands'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]


            # Convert applicable columns to float
            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)


            # Identify relevant columns for levels and filter data
            level_check = set(["Region", "Channel", "Market"])
            existant_cols = list(set(t_filtered.columns) & level_check)
            decider_dic = {"Region": regions, "Channel": channels, "Market": markets}

            key_name = f"{k} | {area_by}" if area_by else k

            # If there is a single level (Region, Channel, or Market)
            if len(existant_cols) == 1:
                filtered_df = t_filtered[t_filtered[t_filtered.columns[1]].isin(    [existant_cols[0]])]
                if not filtered_df.empty:

                    # Rename the existing column to 'Retailer'
                    filtered_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
                    filtered_df['Labels'] = filtered_df['Top Brands'] + "/" + filtered_df[f"{area_by}"]  
                    modified_data[key_name] = pd.concat([modified_data.get(key_name, pd.DataFrame()), filtered_df], ignore_index=True)
                    modified_data[key_name] = modified_data[key_name].sort_values('Value Sales P12M', ascending=False)
                    modified_data[key_name] = modified_data[key_name].drop_duplicates()


            # If there are two levels
            elif len(existant_cols) == 2:
                levels_rank = {"Region": 1, "Channel": 2, "Market": 3}
                existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])

                child_df = t_filtered[t_filtered[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns=[existant_cols[0]])
                parent_df = t_filtered[t_filtered[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns=[existant_cols[1]])
                parent_df[existant_cols[0]] = parent_df[existant_cols[0]].str.replace(" Total", "").str.strip()

                # Rename columns and append to the combined DataFrame
                if not child_df.empty:
                    # child_df.rename(columns={existant_cols[1]: f"{area_by}"}, inplace=True)
                    child_df['Labels'] = child_df['Top Brands'] + "/" + child_df[existant_cols[1]]
                    modified_data[key_name + f" | {existant_cols[1]}"] = child_df
                    modified_data[key_name + f" | {existant_cols[1]}"] = modified_data[key_name + f" | {existant_cols[1]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[key_name + f" | {existant_cols[1]}"] = modified_data[key_name + f" | {existant_cols[1]}"].drop_duplicates()


                if not parent_df.empty:
                    # parent_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
                    parent_df['Labels'] = parent_df['Top Brands'] + "/" + parent_df[existant_cols[0]]  
                    modified_data[key_name + f" | {existant_cols[0]}"] = parent_df
                    modified_data[key_name + f" | {existant_cols[0]}"] = modified_data[key_name + f" | {existant_cols[0]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[key_name + f" | {existant_cols[0]}"] = modified_data[key_name + f" | {existant_cols[0]}"].drop_duplicates()




            # If there are three levels
            elif len(existant_cols) == 3:
                levels_rank = {"Region": 1, "Channel": 2, "Market": 3}
                existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])

                market_df = t_filtered[t_filtered[existant_cols[2]].isin(decider_dic[existant_cols[2]])].drop(columns=[existant_cols[0], existant_cols[1]])
                channel_df = t_filtered[t_filtered[existant_cols[1]].isin([i + " Total" for i in decider_dic[existant_cols[1]]])].drop(columns=[existant_cols[0], existant_cols[2]])
                channel_df[existant_cols[1]] = channel_df[existant_cols[1]].str.replace(" Total", "").str.strip()
                region_df = t_filtered[t_filtered[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns=[existant_cols[1], existant_cols[2]])
                region_df[existant_cols[0]] = region_df[existant_cols[0]].str.replace(" Total", "").str.strip()

                # Rename columns and append to the combined DataFrame
                if not market_df.empty:
                    # market_df.rename(columns={existant_cols[2]: f"{area_by}"}, inplace=True)
                    market_df['Labels'] = market_df['Top Brands'] + "/" + market_df[existant_cols[2]]  

                    modified_data[key_name + f" | {existant_cols[2]}"] = market_df
                    modified_data[key_name + f" | {existant_cols[2]}"] = modified_data[key_name + f" | {existant_cols[2]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[key_name + f" | {existant_cols[2]}"] = modified_data[key_name + f" | {existant_cols[2]}"].drop_duplicates()



                if not channel_df.empty:
                    # channel_df.rename(columns={existant_cols[1]: f"{area_by}"}, inplace=True)
                    channel_df['Labels'] = channel_df['Top Brands'] + "/" + channel_df[existant_cols[1]]  
                    modified_data[key_name + f" | {existant_cols[1]}"] = channel_df
                    modified_data[key_name + f" | {existant_cols[1]}"] = modified_data[key_name + f" | {existant_cols[1]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[key_name + f" | {existant_cols[1]}"] = modified_data[key_name + f" | {existant_cols[1]}"].drop_duplicates()



                if not region_df.empty:
                    # region_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
                    region_df['Labels'] = region_df['Top Brands'] + "/" + region_df[existant_cols[0]]  
                    modified_data[key_name + f" | {existant_cols[0]}"] = region_df
                    modified_data[key_name + f" | {existant_cols[0]}"] = modified_data[key_name + f" | {existant_cols[0]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[key_name + f" | {existant_cols[0]}"] = modified_data[key_name + f" | {existant_cols[0]}"].drop_duplicates()



    return modified_data

In [ ]:
def process_RET_CHAN_Momentum_Sector(dataframes, regions, channels, markets, client_brands, area_by="", slide_for = "",slide_for_list=[]):
    # Dictionary to hold all modified DataFrames in a single structure
    modified_data = {}

    for k, df in dataframes.items():
        # Copy and process each DataFrame
        t = df.copy()

        t.iloc[15] = t.iloc[15].ffill().replace(np.nan, "") 
        t.columns = (t.iloc[16] +" "+ t.iloc[15]).str.strip()
        t = t.iloc[17:]
        if not t.empty:
            t['Top Brands'] = t['Top Brands'].ffill()
            
            top_brands_totals = t[(t['Top Brands'].str.contains("Total")) & (t['Top Brands']!="Grand Total") & (~t["Top Brands"].isin([i+" Total" for i in client_brands]))]

            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)['Top Brands'].to_list()
            competitors = [i.replace(" Total","") for i in competitors]

            level_check = set(["Region", "Channel", "Market"])
            existant_cols = list(set(t.columns) & level_check)

            for col in existant_cols:
                t[col] = t[col].ffill()


            t = t[t[f"{slide_for}"].isin(slide_for_list)]
            t['Sector'] = t['Sector'].ffill()


            t_filtered = t[t['Top Brands'].isin(client_brands) |
                                    ((t['Top Brands'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]


            # Convert applicable columns to float
            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)

            # # Identify relevant columns for levels and filter data
            # level_check = set(["Region", "Channel", "Market"])
            # existant_cols = list(set(t_filtered.columns) & level_check)
            decider_dic = {"Region": regions, "Channel": channels, "Market": markets}

            # If there is a single level (Region, Channel, or Market)
            if len(existant_cols) == 1:
                t_filtered[t_filtered.columns[1]] = t_filtered[t_filtered.columns[1]].ffill()
                filtered_df = t_filtered[t_filtered[t_filtered.columns[1]].isin(decider_dic[existant_cols[0]])]
                if not filtered_df.empty:

                    # Rename the existing column to 'Retailer'
                    filtered_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
                    filtered_df['Labels'] = filtered_df['Top Brands'] + "/" + filtered_df[f"{area_by}"]+ "/" + filtered_df["Sector"]+ "/" + filtered_df[f"{slide_for}"]  
                    modified_data[k] = pd.concat([modified_data.get(k, pd.DataFrame()), filtered_df], ignore_index=True)
                    modified_data[k] = modified_data[k].sort_values('Value Sales P12M', ascending=False)
                    modified_data[k] = modified_data[k].drop_duplicates()



            # If there are two levels
            elif len(existant_cols) == 2:
                levels_rank = {"Region": 1, "Channel": 2, "Market": 3}
                existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])

                t_filtered[existant_cols[1]] = t_filtered[existant_cols[1]].ffill()
                t_filtered[existant_cols[0]] = t_filtered[existant_cols[0]].ffill()

                child_df = t_filtered[t_filtered[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns=[existant_cols[0]])
                parent_df = t_filtered[t_filtered[existant_cols[0]].isin(decider_dic[existant_cols[0]])].drop(columns=[existant_cols[1]])
                # parent_df[existant_cols[0]] = parent_df[existant_cols[0]].str.replace(" Total", "").str.strip()

                # Rename columns and append to the combined DataFrame
                if not child_df.empty:
                    # child_df.rename(columns={existant_cols[1]: f"{area_by}"}, inplace=True)
                    child_df['Labels'] = child_df['Top Brands'] + "/" + child_df[existant_cols[1]]+ "/" + child_df["Sector"]+ "/" + child_df[f"{slide_for}"]  
                    modified_data[k+ f" | {existant_cols[1]}"] = child_df
                    modified_data[k+ f" | {existant_cols[1]}"] = modified_data[k+ f" | {existant_cols[1]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[k+ f" | {existant_cols[1]}"] = modified_data[k+ f" | {existant_cols[1]}"].drop_duplicates()

                if not parent_df.empty:
                    # parent_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
                    parent_df['Labels'] = parent_df['Top Brands'] + "/" + parent_df[existant_cols[0]]+ "/" + parent_df["Sector"]+ "/" + parent_df[f"{slide_for}"]  
                    modified_data[k+ f" | {existant_cols[0]}"] = parent_df
                    modified_data[k+ f" | {existant_cols[0]}"] = modified_data[k+ f" | {existant_cols[0]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[k+ f" | {existant_cols[0]}"] = modified_data[k+ f" | {existant_cols[0]}"].drop_duplicates()


            # If there are three levels
            elif len(existant_cols) == 3:
                levels_rank = {"Region": 1, "Channel": 2, "Market": 3}
                existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])

                t_filtered[existant_cols[2]] = t_filtered[existant_cols[2]].ffill()
                t_filtered[existant_cols[1]] = t_filtered[existant_cols[1]].ffill()
                t_filtered[existant_cols[0]] = t_filtered[existant_cols[0]].ffill()

                market_df = t_filtered[t_filtered[existant_cols[2]].isin(decider_dic[existant_cols[2]])].drop(columns=[existant_cols[0], existant_cols[1]])
                channel_df = t_filtered[t_filtered[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns=[existant_cols[0], existant_cols[2]])
                # channel_df[existant_cols[1]] = channel_df[existant_cols[1]].str.replace(" Total", "").str.strip()
                region_df = t_filtered[t_filtered[existant_cols[0]].isin(decider_dic[existant_cols[0]])].drop(columns=[existant_cols[1], existant_cols[2]])
                # region_df[existant_cols[0]] = region_df[existant_cols[0]].str.replace(" Total", "").str.strip()

                # Rename columns and append to the combined DataFrame
                if not market_df.empty:
                    # market_df.rename(columns={existant_cols[2]: f"{area_by}"}, inplace=True)
                    market_df['Labels'] = market_df['Top Brands'] + "/" + market_df[existant_cols[2]]+ "/" + market_df["Sector"]+ "/" + market_df[f"{slide_for}"]  

                    modified_data[k+ f" | {existant_cols[2]}"] = market_df
                    modified_data[k+ f" | {existant_cols[2]}"] = modified_data[k+ f" | {existant_cols[2]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[k+ f" | {existant_cols[2]}"] = modified_data[k+ f" | {existant_cols[2]}"].drop_duplicates()



                if not channel_df.empty:
                    channel_df.rename(columns={existant_cols[1]: f"{area_by}"}, inplace=True)
                    channel_df['Labels'] = channel_df['Top Brands'] + "/" + channel_df[existant_cols[1]]+ "/" + channel_df["Sector"]+ "/" + channel_df[f"{slide_for}"]  
                    modified_data[k+ f" | {existant_cols[1]}"] = channel_df
                    modified_data[k+ f" | {existant_cols[1]}"] = modified_data[k+ f" | {existant_cols[1]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[k+ f" | {existant_cols[1]}"] = modified_data[k+ f" | {existant_cols[1]}"].drop_duplicates()



                if not region_df.empty:
                    # region_df.rename(columns={existant_cols[0]: f"{area_by}"}, inplace=True)
                    region_df['Labels'] = region_df['Top Brands'] + "/" + region_df[existant_cols[0]]+ "/" + region_df["Sector"]+ "/" + region_df[f"{slide_for}"]  
                    modified_data[k+ f" | {existant_cols[0]}"] = region_df
                    modified_data[k+ f" | {existant_cols[0]}"] = modified_data[k+ f" | {existant_cols[0]}"].sort_values('Value Sales P12M', ascending=False)
                    modified_data[k+ f" | {existant_cols[0]}"] = modified_data[k+ f" | {existant_cols[0]}"].drop_duplicates()



    return modified_data